##  PUC - MG
##  Ciência de Dados e Big Data - 2019
##  TCC - Gilberto Klingen

# Fontes de Dados :

Resultados dos jogos : Free ATP and WTA Results and Stats Databases

https://www.tennisabstract.com/blog/2015/03/24/free-atp-and-wta-results-and-stats-databases/

Probabilidade de vitória de cada jogador em cada partida segundo alfuns sites de apostas : Tennis-Data.co.uk

http://www.tennis-data.co.uk/alldata.php




## Aquisição de Dados

In [1]:
import requests
import pandas as pd
import numpy as np
import xlrd
from datetime import datetime, date
from io import StringIO
from IPython.display import display
from zipfile import ZipFile
import os
from bs4 import BeautifulSoup
import wget
import warnings


# Funções

In [2]:
def get_file(url:str, save_path:str, file_name:str):
    r = requests.get(url)

    with open(save_path + file_name, 'wb') as f:
         f.write(r.content)

In [3]:
def unzip_file (filename:str, directory:str):
    with ZipFile(filename, 'r') as zip_ref:
         zip_ref.extractall(directory)

In [4]:
def wget_file(url:str):
    file = wget.download(url)  
    print(file)

In [5]:
def obtem_arquivos_dos_links(url:str) -> list:    
    lista_arquivos = []
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    for link in soup.find_all("a"):
        if (format(link.get("href"))[-4:] == 'csv' or format(link.get("href"))[-3:] == 'zip'):
           lista_arquivos.append(format(link.get("href"))) 
    return lista_arquivos   

In [6]:
def obtem_arquivos_dos_urls(url:str) -> list:    
    lista_arquivos = []
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    for link in soup.find_all('td id'):
        lista_arquivos.append(format(link.get('class="blob-code blob-code-inner js-file-line"'))) 
    return lista_arquivos   

In [7]:
def grava_dataframe_retorno_url_2(conteudo_arquivo:str) -> pd.DataFrame:
    dfdata = StringIO(conteudo_arquivo)
    df = pd.read_csv(dfdata, sep=";")
    # print(df)
    # display(df)
    return df

In [8]:
def grava_dataframe_retorno_url(conteudo_arquivo:pd.DataFrame) -> pd.DataFrame:
    dfdata = conteudo_arquivo[2]
    df = pd.read_csv(dfdata, sep=";")
    # print(df)
    # display(df)
    return df

In [9]:
def obtem_nome_abreviado_text(nome:str) -> str:
    # de , di, van der, van
    
    partes_nome=nome.split(' ')
    
    if 'di' in partes_nome:
        separator_index = partes_nome.index('di')
    elif 'de' in partes_nome:          
          separator_index = partes_nome.index('de')       
    elif 'van' in partes_nome:          
          separator_index = partes_nome.index('van')       
    else:
          separator_index = partes_nome.size                      
                             
    nome = ''
    for ind in range(0,separator_index):
        nome = nome + ' ' + partes_nome[ind][0] + '.'
    nome = nome.strip()    
    
    sobrenome = ' '.join(partes_nome[separator_index:])
    
    return sobrenome + ' ' + nome

In [10]:
def obtem_nome_abreviado(lst_nome:list) -> list:
    # de , di, van der, van
    
    lst_return = []
    
    for nome in lst_nome:
        partes_nome=str(nome).split(' ')
    
        if 'di' in partes_nome:
           separator_index = partes_nome.index('di')
        elif 'de' in partes_nome:          
          separator_index = partes_nome.index('de')       
        elif 'van' in partes_nome:          
          separator_index = partes_nome.index('van')       
        else:
          separator_index = len(partes_nome) - 1                      
                             
        nome = ''
        for ind in range(0,separator_index):
            nome = nome + ' ' + partes_nome[ind][0] + '.'
        nome = nome.strip()    
    
        sobrenome = ' '.join(partes_nome[separator_index:])
    
        lst_return.append(sobrenome + ' ' + nome) 
    
    return lst_return

In [11]:
def insere_colunas_odds_medios(df_odds:pd.DataFrame):

    lst_sites = ['CB','B365','GB','EX','IW','LB','SB','PS','SJ','B&W','UB']
    lst_w_sites = []
    lst_l_sites = []
    
#   Adiciona as colunas de odds medios (AvgW e AvgL)
    
    df_odds['AvgW'] = np.nan  
    df_odds['AvgL'] = np.nan  
     
#   Monta lista de colunas existentes para o ano
    
    for site in lst_sites:
        column = site + 'W'
        if column in df_odds:
           lst_w_sites.append(column)
        
        column = site + 'L'
        if column in df_odds:
           lst_l_sites.append(column)
        
#   Calcula os odds medios        
    
    for ind in df_odds.index:
        
        count_w = 0
        count_l = 0
    
        sum_w = 0.0
        sum_l = 0.0
        
        for site in lst_w_sites:
            
            # site_l = site[0:-1] + 'L'

            # if df_odds.loc[ind,site] > df_odds.loc[ind,site_l]:
            #   # print('{} / {}'.format(df_odds.loc[ind,site], df_odds.loc[ind,site_l])) 
            #   aux_swap = df_odds.loc[ind,site]
            #   df_odds.iloc[ind, df_odds.columns.get_loc(site)] =  df_odds.loc[ind,site_l]
            #   df_odds.iloc[ind, df_odds.columns.get_loc(site_l)]  = aux_swap 
                
            if not np.isnan(df_odds.loc[ind,site]):
               count_w = count_w + 1
               sum_w = sum_w + df_odds.loc[ind,site]
                
        for site in lst_l_sites:
            if not np.isnan(df_odds.loc[ind,site]):
               count_l = count_l + 1
               sum_l = sum_l + df_odds.loc[ind,site]    
                
        if count_w != 0:        
           df_odds.iloc[ind, df_odds.columns.get_loc('AvgW')] = round(sum_w / count_w,2)
        if count_l != 0:     
           df_odds.iloc[ind, df_odds.columns.get_loc('AvgL')] = round(sum_l / count_l,2)

In [12]:
def obtem_dataframe_resultados_bkp(ano:str) -> pd.DataFrame:
    URL="https://github.com/JeffSackmann/tennis_atp/blob/master/atp_matches_{}.csv".format(ano)
    
    # Obtem o HTML relativo a URL
    html_content = requests.get(URL).text
    
    # Obtem as tabelas contidas no HTML
    html_tables = pd.read_html(html_content)
    
    # Gera um dataframe contendo os dados de resultados
    df0 = html_tables[0]
    
    # Gera um dataframe eliminando a coluna 0 (NaN) e renomeando a coluna 1 para "Texto"
    df1=df0.iloc[:,1:2].rename(columns={1:'Texto'})
    
    # Gera um dataframe transformando a coluna "Texto" em colunas deste novo dataframe
    df2 = df1.Texto.str.split(",",expand=True) 
    
    # Gera uma listas contendo os nomes das colunas que estão na linha 0 do dataframe df2
    lst_col=df2.iloc[0].tolist()
    # print(lst_col)
    
    # Gera um dataframe eliminando a primeira linha de df2 (lista de nomes das colunas)
    df3 = df2[1:]
    
    # Renomeia as colunas de df3 de acordo com a lista (lst_col) obtida anteriormente
    df3.columns = lst_col

    df3['winner_name']= df3['winner_name'].str.strip()
    df3['loser_name']= df3['loser_name'].str.strip()
    
    # adiciona a coluna Year (Ano)
    df3['Year']=int(ano)
    
    # altera o tipoda coluna match_num para inteiro
    df3['match_num'] = df3['match_num'].astype(int)
    
    # ordena o dataframe por data e identificação do jogo
    df3.sort_values(['tourney_date','match_num'],inplace=True)
    # print(df3)
    

    
    return df3

In [13]:
def obtem_dataframe_resultados(ano:str) -> pd.DataFrame:
    URL="https://github.com/JeffSackmann/tennis_atp/blob/master/atp_matches_{}.csv".format(ano)
    
    # Obtem o HTML relativo a URL
    html_content = requests.get(URL).text
    
    # Obtem as tabelas contidas no HTML
    html_tables = pd.read_html(html_content)
    
    # Gera um dataframe contendo os dados de resultados
    df0 = html_tables[0]
    
    if ano == '2020':
       df0.drop("Unnamed: 0", axis=1, inplace=True)
       df3 = df0
    else:    
    
    # Gera um dataframe eliminando a coluna 0 (NaN) e renomeando a coluna 1 para "Texto"
       df1=df0.iloc[:,1:2].rename(columns={1:'Texto'})
    
    # Gera um dataframe transformando a coluna "Texto" em colunas deste novo dataframe
       df2 = df1.Texto.str.split(",",expand=True) 
    
    # Gera uma listas contendo os nomes das colunas que estão na linha 0 do dataframe df2
       lst_col=df2.iloc[0].tolist()
    # print(lst_col)
    
    # Gera um dataframe eliminando a primeira linha de df2 (lista de nomes das colunas)
       df3 = df2[1:]
    
    # Renomeia as colunas de df3 de acordo com a lista (lst_col) obtida anteriormente
       df3.columns = lst_col

    df3['winner_name']= df3['winner_name'].str.strip()
    df3['loser_name']= df3['loser_name'].str.strip()
    
    # adiciona a coluna Year (Ano)
    df3['Year']=int(ano)
    
    # altera o tipoda coluna match_num para inteiro
    df3['match_num'] = df3['match_num'].astype(int)
    
    # ordena o dataframe por data e identificação do jogo
    df3.sort_values(['tourney_date','match_num'],inplace=True)
    # print(df3)
    

    
    return df3

In [14]:
def obtem_dataframe_ODDS(ano:str) -> pd.DataFrame:
    
    # TRIM(jogador), "." na ultima posicao
    URL='http://www.tennis-data.co.uk/{}/{}.zip'.format(ano,ano)
    
    get_file(URL, '/Users/gilberto/Downloads/', 'ODDS_{}.zip'.format(ano))
    
    lst_files = ZipFile('/Users/gilberto/Downloads/ODDS_{}.zip'.format(ano)).namelist()
    
    unzip_file ('/Users/gilberto/Downloads/ODDS_{}.zip'.format(ano),'/Users/gilberto/Downloads/')
    
    df = pd.read_excel('/Users/gilberto/Downloads/' + lst_files[0], sheet_name=ano, skiprows = 0, index_col=None) 
    
    # adiciona a coluna Year (Ano)
    df['Year']=int(ano)
    
    # ===============
    # Limpeza da base
    # ===============
    
    # Retira caracteres em branco do nome do vencedor e do perdedor
    # Adiciona um "." ao final de cada nome do vencedor e do perdedor
    # substitui no nome do vencedor e do perdedor ".." por "." desta forma garante que todos os nomes terminem com "."
    
    df['Winner']= df['Winner'].str.strip()
    df['Winner']= df['Winner'].map(str) + '.'
    df['Winner'].replace('\.\.','.', inplace=True, regex=True)
    
    df['Loser']= df['Loser'].str.strip()
    df['Loser']= df['Loser'].map(str) + '.'
    df['Loser'].replace('\.\.','.', inplace=True, regex=True)
    
    if ano < '2010':
       insere_colunas_odds_medios(df) 
    
    # ===========================================================================================
    # Insere colunas de identificação de torneios e jogadores para permitir futuramente a junção
    # com a base de resultados da ATP
    # ===========================================================================================
    
    # df.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\ODDS_XXX_2001.csv', index = False)
    
    # dfx = df.loc[df['Winner'].isin(['Bahrouzyan O.','Gromley C.','Hantschek M.','Kutac R.','Luncanu P.A.','Marin L.','Stepanek M.'])]
    # dfy = df.loc[df['Loser'].isin(['Bahrouzyan O.','Gromley C.','Hantschek M.','Kutac R.','Luncanu P.A.','Marin L.','Stepanek M.'])]

    # dfx=pd.concat([dfx,dfy])
    # print(dfx)
    
    # dfx.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\ODDS_ZZZZ_' + str(ano) + '.csv', index = False)

    return df

In [15]:
def gera_lista_jogadores_arquivo_ODDS_em_EXCEL():
    set_players = set()
    
    for ano in range(2001,2020):
        print(ano)
        
        df_odds = obtem_dataframe_ODDS(str(ano))
        w_set_players = set(df_odds.Winner)
        l_set_players = set(df_odds.Loser)
        set_players=set_players.union(w_set_players).union(l_set_players)
        
    df_players = pd.DataFrame(sorted(set_players))    
    df_players.to_excel("C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\players_odds.xlsx")  

In [16]:
def gera_lista_jogadores_arquivo_results_em_EXCEL():
    df_players = pd.DataFrame(columns=['player_id','player_name','player_abrev_name'])
    for ano in range(2001,2020):
        print(ano)
        
        df_results = obtem_dataframe_resultados(str(ano))
        
        df_winners_year = df_results[['winner_id', 'winner_name']].copy()
        df_winners_year.rename(columns={'winner_id': 'player_id', 'winner_name': 'player_name'}, inplace=True)
        
        df_players = pd.concat([df_winners_year, df_players],ignore_index=True).drop_duplicates().reset_index(drop=True)
        
        df_losers_year = df_results[['loser_id', 'loser_name']].copy()
        df_losers_year.rename(columns={'loser_id': 'player_id', 'loser_name': 'player_name'}, inplace=True)
                
        df_players = pd.concat([df_losers_year, df_players],ignore_index=True).drop_duplicates().reset_index(drop=True)
        
    # print(obtem_nome_abreviado(list(df_players['player_name'])))    
    df_players['player_abrev_name'] = obtem_nome_abreviado(list(df_players['player_name']))       
    df_players.to_excel("C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\players_results.xlsx")    
    
    print('OK')

In [17]:
def gera_lista_torneios_arquivo_ODDS_em_EXCEL():
    
    for ano in range(2001,2020):
        print(ano)
        
        df_odds = obtem_dataframe_ODDS(str(ano))
        df_tourneys = df_odds[['Year', 'ATP', 'Location', 'Tournament']].copy()
        df_tourneys.drop_duplicates(keep = 'first', inplace = True) 
        
        if ano == 2001:
           df_all_tourneys = df_tourneys
        else:
           df_all_tourneys = pd.concat([df_all_tourneys,df_tourneys],ignore_index=True).drop_duplicates().reset_index(drop=True)
    
    # print(df_all_tourneys)
    df_all_tourneys.to_excel("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\tourneys_odds.xlsx")  

In [18]:
def gera_lista_torneios_arquivo_results_em_EXCEL():
    
    for ano in range(2001,2020):
        print(ano)
        
        df_results = obtem_dataframe_resultados(str(ano))
        df_tourneys = df_results[['Year', 'tourney_id', 'tourney_name']].copy()
        df_tourneys.drop_duplicates(keep = 'first', inplace = True) 
        
        if ano == 2001:
           df_all_tourneys = df_tourneys
        else:
           df_all_tourneys = pd.concat([df_all_tourneys,df_tourneys],ignore_index=True).drop_duplicates().reset_index(drop=True)
    
    #print(df_all_tourneys)
    df_all_tourneys.to_excel("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\tourneys_results.xlsx")  

In [19]:
def union_df_unknown_players():
    for ano in range(2001,2020):
        df = pd.read_csv("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\ODDS_ZZZZ_" + str(ano) + ".csv", sep=',', delimiter=None, header='infer', names=None, index_col=None)
        dfx = df[['Year', 'Location', 'Tournament' , 'Date', 'Winner', 'Loser', 'W1','L1','W2','L2','W3','L3','W4','L4','W5', 'L5']].copy()
        if ano == 2001:
           dfy = dfx
        else:
           dfy=pd.concat([dfy,dfx])  
    print(dfy)    
    dfy.to_excel("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\crazy_players.xlsx")

In [20]:
def merge_atp_and_odds_info_old():
    
    warnings.filterwarnings("ignore")
    
    # Obtem os dataframes de de=para
    # ------------------------------
    
    df_de_para_players = pd.read_excel('C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\TCC\\Meu_Trabalho\\players_de_para.xlsx', sheet_name='DE_PARA', skiprows = 0, index_col=None)
    df_de_para_tourneys = pd.read_excel('C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\TCC\\Meu_Trabalho\\tourneys_de_para.xlsx', sheet_name='DE_PARA', skiprows = 0, index_col=None)
    df_de_para_round = pd.read_excel('C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\TCC\\Meu_Trabalho\\round_de_para.xlsx', sheet_name='DE_PARA', skiprows = 0, index_col=None)
    
    for ano in range(2000,2020):
        
        print(' ')
        print(str(ano))
        
        # Obtem o winner_id
        # -----------------
        
        df_odds = obtem_dataframe_ODDS(str(ano))
        
        df_1 = df_odds.merge(df_de_para_players, how='left', left_on=['Winner'], right_on=['Player_Name'])
        
        df_NaN = df_1[df_1['Player_ID'].isnull()]
        
        if not df_NaN.empty:
           print('Lista de winners contém nulos') 
           print(df_NaN['Winner'] + ' ' + df_NaN['Loser'])
           break
        
        df_1.rename(columns={'Player_ID': 'winner_id'}, inplace=True)
        
        # Obtem o loser_id
        # -----------------
        
        df_2 = df_1.merge(df_de_para_players, how='left', left_on=['Loser'], right_on=['Player_Name'])
        
        df_NaN = df_2[df_2['Player_ID'].isnull()]
        
        if not df_NaN.empty:
           print('Lista de losers contém nulos') 
           print(df_NaN['Winner'] + ' ' + df_NaN['Loser'])
           break
        
        df_2.rename(columns={'Player_ID': 'loser_id'}, inplace=True)
                
        # Merge com de-para de torneios
        # -----------------------------
        
        df_3 = df_2.merge(df_de_para_tourneys, how='left', left_on=['Year', 'ATP', 'Location','Tournament'], right_on=['Year', 'ATP', 'Location','Tournament'])
        
        df_3['tourney_id'] = df_3['tourney_id'].astype("string")
        df_3['Round'] = df_3['Round'].astype("string")
        
        # dataTypeSeries = df_3.dtypes
        # print('Data type of each column of Dataframe :')
        # print(dataTypeSeries)

        df_NaN = df_3[df_3['tourney_id'].isnull()]
        
        if not df_NaN.empty:
           print('Lista de tourneys contém nulos') 
           print(df_NaN)
           break
        
        # Obtem dataframe de resultados ATP
        # ---------------------------------
        
        df_results = obtem_dataframe_resultados(str(ano))
        
        df_results['round'] = df_results['round'].astype("string")
        df_results['draw_size'] = df_results['draw_size'].astype(int)
        
        df_4 = df_results.merge(df_de_para_round, how='left', left_on=['draw_size', 'round'], right_on=['draw_size', 'round'])
        
        df_NaN = df_4[df_4['round_odds'].isnull()]
        
        if not df_NaN.empty:
           print('Lista de rounds contém nulos') 
           print(df_NaN)
           break
        
        df_4['tourney_id'] = df_4['tourney_id'].astype("string")
        df_4['winner_id'] = df_4['winner_id'].astype(int)
        df_4['loser_id'] = df_4['loser_id'].astype(int)
        df_4['round_odds'] = df_4['round_odds'].astype("string")

        # dataTypeSeries = df_4.dtypes
        # print('Data type of each column of Dataframe :')
        # print(dataTypeSeries)
        
        # df_merged_3_4 =  df_3.merge(df_4, how='left', left_on=['Year', 'tourney_id', 'Round', 'winner_id','loser_id'], right_on=['Year', 'tourney_id', 'round_odds', 'winner_id','loser_id'])
        
        # df_NaN = df_merged_3_4[df_merged_3_4['draw_size'].isnull()]
        
        # if not df_NaN.empty:
        #   print('Lista de resultados contém nulos') 
        #   print(df_NaN)
        #   df_NaN.to_excel("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\merged_3_4_NAN.xlsx")

        df_merged_4_3 =  df_4.merge(df_3, how='left', right_on=['Year', 'tourney_id', 'Round', 'winner_id','loser_id'], left_on=['Year', 'tourney_id', 'round_odds', 'winner_id','loser_id'])
        
        df_merged_4_3.to_excel("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\merged_ATP_ODDS.xlsx")
                
        print('OK')
        
        # df_NaN = df_merged_4_3[df_merged_4_3['W1'].isnull()]
        
        # if not df_NaN.empty:
        #   print('Lista de odds contém nulos') 
        #   print(df_NaN)
        #   df_NaN.to_excel("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\merged_4_3_NAN.xlsx")
            
        # print(df_merged)

In [21]:
def merge_atp_and_odds_info():
    
    warnings.filterwarnings("ignore")
    
    df_merged = pd.DataFrame
    
    # Obtem os dataframes de de=para
    # ------------------------------
    
    df_de_para_players = pd.read_excel('C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\TCC\\Meu_Trabalho\\players_de_para.xlsx', sheet_name='DE_PARA', skiprows = 0, index_col=None)
    df_de_para_tourneys = pd.read_excel('C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\TCC\\Meu_Trabalho\\tourneys_de_para.xlsx', sheet_name='DE_PARA', skiprows = 0, index_col=None)
    df_de_para_round = pd.read_excel('C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\TCC\\Meu_Trabalho\\round_de_para.xlsx', sheet_name='DE_PARA', skiprows = 0, index_col=None)
    
    for ano in range(2000,2020):
        
        # print(' ')
        print(str(ano))
        
        df_odds = obtem_dataframe_ODDS(str(ano))
            
        # Obtem o winner_id
        # -----------------
                
        df_1 = df_odds.merge(df_de_para_players, how='left', left_on=['Winner'], right_on=['Player_Name'])
        
        df_1.rename(columns={'Player_ID': 'winner_id'}, inplace=True)
        
        # Obtem o loser_id
        # -----------------
        
        df_2 = df_1.merge(df_de_para_players, how='left', left_on=['Loser'], right_on=['Player_Name'])
       
        df_2.rename(columns={'Player_ID': 'loser_id'}, inplace=True)
                
        # Merge com de-para de torneios
        # -----------------------------
        
        df_3 = df_2.merge(df_de_para_tourneys, how='left', left_on=['Year', 'ATP', 'Location','Tournament'], right_on=['Year', 'ATP', 'Location','Tournament'])
        
        df_3['tourney_id'] = df_3['tourney_id'].astype("string")
        df_3['Round'] = df_3['Round'].astype("string")
        
        # Obtem dataframe de resultados ATP
        # ---------------------------------
        
        df_results = obtem_dataframe_resultados(str(ano))
        
        df_results['round'] = df_results['round'].astype("string")
        df_results['draw_size'] = df_results['draw_size'].astype(int)
        
        df_4 = df_results.merge(df_de_para_round, how='left', left_on=['draw_size', 'round'], right_on=['draw_size', 'round'])
        
        df_4['tourney_id'] = df_4['tourney_id'].astype("string")
        df_4['winner_id'] = df_4['winner_id'].astype(int)
        df_4['loser_id'] = df_4['loser_id'].astype(int)
        df_4['round_odds'] = df_4['round_odds'].astype("string")

        df_merged_4_3 =  df_4.merge(df_3, how='left', right_on=['Year', 'tourney_id', 'Round', 'winner_id','loser_id'], left_on=['Year', 'tourney_id', 'round_odds', 'winner_id','loser_id'])
        
        if not df_merged.empty: 
           df_merged = df_merged.append(df_merged_4_3)  
        else:
           df_merged = df_merged_4_3 
        
    # Retira os jogos da Taça Davis pois para esses jogos não há apostas
    # ------------------------------------------------------------------
    
    df_merged = df_merged[df_merged['tourney_level'] != 'D']
                          
    df_merged.to_excel("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\merged_ATP_ODDS.xlsx")
                
    print('OK')

In [22]:
def gera_dataframes_resultados_agregados():
    
    warnings.filterwarnings("ignore")
    
    # cria dataframe de dados agregados por jogador
    
    columns_1 = {
                 'player_id':'',
                 'tourney_id':'',
                 'tourney_date':'',
                 'round':'',
                 'match_num':int,
                 'match_seq':int,
                 'qty_grass_matches':int,
                 'qty_grass_victories':int,
                 'pct_grass_victories':float,
                 'qty_hard_matches':int,
                 'qty_hard_victories':int,
                 'pct_hard_victories':float,
                 'qty_clay_matches':int,
                 'qty_clay_victories':int,
                 'pct_clay_victories':float,
                 'qty_matches_against_left_handed':int,
                 'qty_victories_against_left_handed':int,
                 'pct_victories_against_left_handed':float,
                 'qty_3sets_matches':int,
                 'qty_3sets_victories':int,
                 'pct_3sets_victories':float,
                 'qty_5sets_matches':int,
                 'qty_5sets_victories':int,
                 'pct_5sets_victories':float,
                 'minutes':int,
                 'avg_minutes':float,
                 'qty_ace':int,
                 'qty_df':int,
                 'qty_svpt':int,
                 'qty_1stIn':int,
                 'qty_1stWon':int,
                 'qty_2ndWon':int,
                 'qty_SvGms':int,
                 'qty_bpSaved':int,
                 'qty_bpFaced':int,
                 'avg_ace':float,
                 'avg_df':float,
                 'avg_svpt':float,
                 'avg_1stIn':float,
                 'avg_1stWon':float,
                 'avg_2ndWon':float,
                 'avg_SvGms':float,
                 'avg_bpSaved':float,
                 'avg_bpFaced':float,
                 'height':int,
                 'age':float,
                 'rank_points':int,
                 'qty_matches_grand_slam':int,
                 'qty_victories_grand_slam':int,
                 'pct_victories_grand_slam':float,
                 'qty_matches_master_1000':int,
                 'qty_victories_master_1000':int,
                 'pct_victories_master_1000':float,
                 'qty_matches_ATP_250_500':int,
                 'qty_victories_ATP_250_500':int,
                 'pct_victories_ATP_250_500':float,
                 'qty_matches_finals':int,
                 'qty_victories_finals':int,
                 'pct_victories_finals':float,
                 'qty_grand_slam_titles':int,
                 'qty_master_1000_titles':int,
                 'qty_ATP_250_500_titles':int,
                 'qty_finals_titles':int
                 }
    
    df_1_player = pd.DataFrame(columns_1, index=[])
    df_1_player_geral = pd.DataFrame(columns_1, index=[])
    
    # cria dataframe de dados agregados por par de jogadores
    
    columns_2 = {
                 'player_1_id':'',
                 'player_2_id':'',
                 'tourney_id':'',
                 'tourney_date':'',
                 'round':'',
                 'match_num':int,
                 'match_seq':int,
                 'qty_grass_matches':int,
                 'qty_player_1_grass_victories':int,
                 'pct_player_1_grass_victories':float,
                 'qty_hard_matches':int,
                 'qty_player_1_hard_victories':int,
                 'pct_player_1_hard_victories':float,
                 'qty_clay_matches':int,
                 'qty_player_1_clay_victories':int,
                 'pct_player_1_clay_victories':float,
                 'qty_3sets_matches':int,
                 'qty_player_1_3sets_victories':int,
                 'pct_player_1_3sets_victories':float,
                 'qty_5sets_matches':int,
                 'qty_player_1_5sets_victories':int,
                 'pct_player_1_5sets_victories':float,
                 'qty_grand_slam_matches':int,
                 'qty_player_1_grand_slam_victories':int,
                 'pct_player_1_grand_slam_victories':float,
                 'qty_master_1000_matches':int,
                 'qty_player_1_master_1000_victories':int,
                 'pct_player_1_master_1000_victories':float,
                 'qty_finals_matches':int,
                 'qty_player_1_finals_victories':int,
                 'pct_player_1_finals_victories':float,
                 'qty_ATP_250_500_matches':int,
                 'qty_player_1_ATP_250_500_victories':int,
                 'pct_player_1_ATP_250_500_victories':float
                }

    df_2_player = pd.DataFrame(columns_2, index=[])
    df_2_player_geral = pd.DataFrame(columns_2, index=[])
    
    for ano in range(1998,2020):
        
        df_results = obtem_dataframe_resultados(str(ano))
        
        print('--------------------------------------------')
        print(str(ano))
        now = datetime.now()
        print(now.strftime("%H:%M:%S"))
        print(str(len(df_results.index)))
        print('--------------------------------------------')
        contador = 0
        
        for ind in df_results.index:
            
            contador = contador + 1
            
            if (contador % 100) == 1:
               now = datetime.now()
               print(str(contador) + '-' + now.strftime("%H:%M:%S")) 
                        
            #if contador == 101:
            #   break 
            
            # df3x = df3[(df3['winner_id'] == '103720') & (df3['match_num'] == 1)]

            # ################################################ #
            #         Gera dados agregados do vencedor         #
            # ################################################ #
                
            df_1_player_aux_1 = df_1_player[df_1_player['player_id'] == df_results.loc[ind,'winner_id']]
                                                                                        
            if not df_1_player_aux_1.empty:                
               df_1_player_aux_1.sort_values(['match_seq'],inplace=True,ascending=[False])
               df_1_player_aux_1.reset_index(drop=True,inplace=True)
               # df_1_player_aux_1.reset_index(inplace=True)
            
               # print('Winner encontrado *********** :{} '.format(df_results.loc[ind,'winner_id'])) 
            
               df_1_player_aux_1.loc[0,"tourney_id"] = df_results.loc[ind,'tourney_id']
               df_1_player_aux_1.loc[0,"tourney_date"] = df_results.loc[ind,'tourney_date'] 
               df_1_player_aux_1.loc[0,"match_num"] = df_results.loc[ind,'match_num'] 
               df_1_player_aux_1.loc[0,"round"] = df_results.loc[ind,'round']  
               
            else:
               # print('Winner não encontrado :{}'.format(df_results.loc[ind,'winner_id'])) 
                
               df_1_player_aux_1 = pd.DataFrame([[
                                df_results.loc[ind,'winner_id'],
                                df_results.loc[ind,'tourney_id'],  
                                df_results.loc[ind,'tourney_date'],  
                                df_results.loc[ind,'round'],  
                                df_results.loc[ind,'match_num'],
                                0 ,   # match_seq  
                                0 ,   # qty_grass_matches
                                0 ,   # qty_grass_victories
                                0.0 , # pct_grass_victories
                                0 ,   # qty_hard_matches
                                0 ,   # qty_hard_victories
                                0.0 , # pct_hard_victories
                                0 ,   # qty_clay_matches
                                0 ,   # qty_clay_victories
                                0.0 , # pct_clay_victories
                                0 ,   # qty_matches_against_left_handed
                                0 ,   # qty_victories_against_left_handed
                                0.0 , # pct_victories_against_left_handed
                                0 ,   # qty_3sets_matches
                                0 ,   # qty_3sets_victories
                                0.0 , # pct_3sets_victories
                                0 ,   # qty_5sets_matches
                                0 ,   # qty_5sets_victories
                                0.0 , # pct_5sets_victories
                                0   , # minutes
                                0.0 , # avg_minutes
                                0 ,   # qty_ace
                                0 ,   # qty_df
                                0 ,   # qty_svpt
                                0 ,   # qty_1stIn
                                0 ,   # qty_1stWon
                                0 ,   # qty_2ndWon
                                0 ,   # qty_SvGms
                                0 ,   # qty_bpSaved
                                0 ,   # qty_bpFaced
                                0.0 , # avg_ace
                                0.0 , # avg_df
                                0.0 , # avg_svpt
                                0.0 , # avg_1stIn
                                0.0 , # avg_1stWon
                                0.0 , # avg_2ndWon
                                0.0 , # avg_SvGms
                                0.0 , # avg_bpSaved
                                0.0 , # avg_bpFaced
                                0.0 , # height
                                0.0 , # age
                                0   , # rank_points
                                0 ,   # qty_matches_grand_slam
                                0 ,   # qty_victories_grand_slam
                                0.0 , # pct_victories_grand_slam
                                0 ,   # qty_matches_master_1000
                                0 ,   # qty_victories_master_1000
                                0.0 , # pct_victories_master_1000
                                0 ,   # qty_matches_ATP_250_500
                                0 ,   # qty_victories_ATP_250_500
                                0.0 , # pct_victories_ATP_250_500
                                0 ,   # qty_matches_finals
                                0 ,   # qty_victories_finals
                                0.0 , # pct_victories_finals
                                0 ,   # qty_grand_slam_titles
                                0 ,   # qty_master_1000_titles
                                0 ,   # qty_ATP_250_500_titles
                                0     # qty_finals_titles
                                       ]], columns = columns_1 )    
            
            df_1_player_aux_1.loc[0,"match_seq"] = df_1_player_aux_1.loc[0,"match_seq"] + 1
                      
            if df_results.loc[ind,'surface'] == 'Clay':
               df_1_player_aux_1.loc[0,"qty_clay_matches"] = df_1_player_aux_1.loc[0,"qty_clay_matches"] + 1
               df_1_player_aux_1.loc[0,"qty_clay_victories"] = df_1_player_aux_1.loc[0,"qty_clay_victories"] + 1   
               df_1_player_aux_1.loc[0,"pct_clay_victories"] = df_1_player_aux_1.loc[0,"qty_clay_victories"] / df_1_player_aux_1.loc[0,"qty_clay_matches"] * 100     
            elif df_results.loc[ind,'surface'] == 'Grass':   
                 df_1_player_aux_1.loc[0,"qty_grass_matches"] = df_1_player_aux_1.loc[0,"qty_grass_matches"] + 1
                 df_1_player_aux_1.loc[0,"qty_grass_victories"] = df_1_player_aux_1.loc[0,"qty_grass_victories"] + 1   
                 df_1_player_aux_1.loc[0,"pct_grass_victories"] = df_1_player_aux_1.loc[0,"qty_grass_victories"] / df_1_player_aux_1.loc[0,"qty_grass_matches"] * 100  
            else:
                 df_1_player_aux_1.loc[0,"qty_hard_matches"] = df_1_player_aux_1.loc[0,"qty_hard_matches"] + 1 
                 df_1_player_aux_1.loc[0,"qty_hard_victories"] = df_1_player_aux_1.loc[0,"qty_hard_victories"] + 1   
                 df_1_player_aux_1.loc[0,"pct_hard_victories"] = df_1_player_aux_1.loc[0,"qty_hard_victories"] / df_1_player_aux_1.loc[0,"qty_hard_matches"] * 100    
                
            if df_results.loc[ind,'loser_hand'] == 'L':
               df_1_player_aux_1.loc[0,"qty_matches_against_left_handed"] = df_1_player_aux_1.loc[0,"qty_matches_against_left_handed"] + 1
               df_1_player_aux_1.loc[0,"qty_victories_against_left_handed"] = df_1_player_aux_1.loc[0,"qty_victories_against_left_handed"] + 1
               df_1_player_aux_1.loc[0,"pct_victories_against_left_handed"] = df_1_player_aux_1.loc[0,"qty_victories_against_left_handed"] / df_1_player_aux_1.loc[0,"qty_matches_against_left_handed"] * 100  
            
            
            if df_results.loc[ind,'best_of'] == '3':
               df_1_player_aux_1.loc[0,"qty_3sets_matches"] = df_1_player_aux_1.loc[0,"qty_3sets_matches"] + 1
               df_1_player_aux_1.loc[0,"qty_3sets_victories"] = df_1_player_aux_1.loc[0,"qty_3sets_victories"] + 1
               df_1_player_aux_1.loc[0,"pct_3sets_victories"] = df_1_player_aux_1.loc[0,"qty_3sets_victories"] / df_1_player_aux_1.loc[0,"qty_3sets_matches"] * 100   
            else:
               df_1_player_aux_1.loc[0,"qty_5sets_matches"] = df_1_player_aux_1.loc[0,"qty_5sets_matches"] + 1
               df_1_player_aux_1.loc[0,"qty_5sets_victories"] = df_1_player_aux_1.loc[0,"qty_5sets_victories"] + 1
               df_1_player_aux_1.loc[0,"pct_5sets_victories"] = df_1_player_aux_1.loc[0,"qty_5sets_victories"] / df_1_player_aux_1.loc[0,"qty_5sets_matches"] * 100   
 
            df_1_player_aux_1.loc[0,"minutes"] = df_1_player_aux_1.loc[0,"minutes"] + int("0" + df_results.loc[ind,'minutes'])
            df_1_player_aux_1.loc[0,"avg_minutes"] = df_1_player_aux_1.loc[0,"minutes"] / df_1_player_aux_1.loc[0,"match_seq"]
    
            df_1_player_aux_1.loc[0,"qty_ace"] = df_1_player_aux_1.loc[0,"qty_ace"] + int("0" + df_results.loc[ind,'w_ace'])
            df_1_player_aux_1.loc[0,"qty_df"] = df_1_player_aux_1.loc[0,"qty_df"] + int("0" + df_results.loc[ind,'w_df'])
            df_1_player_aux_1.loc[0,"qty_svpt"] = df_1_player_aux_1.loc[0,"qty_svpt"] + int("0" + df_results.loc[ind,'w_svpt'])
            df_1_player_aux_1.loc[0,"qty_1stIn"] = df_1_player_aux_1.loc[0,"qty_1stIn"] + int("0" + df_results.loc[ind,'w_1stIn'])
            df_1_player_aux_1.loc[0,"qty_1stWon"] = df_1_player_aux_1.loc[0,"qty_1stWon"] + int("0" + df_results.loc[ind,'w_1stWon'])
            df_1_player_aux_1.loc[0,"qty_2ndWon"] = df_1_player_aux_1.loc[0,"qty_2ndWon"] + int("0" + df_results.loc[ind,'w_2ndWon'])
            df_1_player_aux_1.loc[0,"qty_SvGms"] = df_1_player_aux_1.loc[0,"qty_SvGms"] + int("0" + df_results.loc[ind,'w_SvGms'])
            df_1_player_aux_1.loc[0,"qty_bpSaved"] = df_1_player_aux_1.loc[0,"qty_bpSaved"] + int("0" + df_results.loc[ind,'w_bpSaved'])
            df_1_player_aux_1.loc[0,"qty_bpFaced"] = df_1_player_aux_1.loc[0,"qty_bpFaced"] + int("0" + df_results.loc[ind,'w_bpFaced'])

            df_1_player_aux_1.loc[0,"avg_ace"] = df_1_player_aux_1.loc[0,"qty_ace"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_df"] = df_1_player_aux_1.loc[0,"qty_df"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_svpt"] = df_1_player_aux_1.loc[0,"qty_svpt"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_1stIn"] = df_1_player_aux_1.loc[0,"qty_1stIn"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_1stWon"] = df_1_player_aux_1.loc[0,"qty_1stWon"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_2ndWon"] = df_1_player_aux_1.loc[0,"qty_2ndWon"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_SvGms"] = df_1_player_aux_1.loc[0,"qty_SvGms"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_bpSaved"] = df_1_player_aux_1.loc[0,"qty_bpSaved"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_bpFaced"] = df_1_player_aux_1.loc[0,"qty_bpFaced"] / df_1_player_aux_1.loc[0,"match_seq"]
 
            df_1_player_aux_1.loc[0,"height"] = int("0" + df_results.loc[ind,'winner_ht'])
            df_1_player_aux_1.loc[0,"age"] =  float("0" + df_results.loc[ind,'winner_age'])
            df_1_player_aux_1.loc[0,"rank_points"] = int("0" + df_results.loc[ind,'winner_rank_points'])
            
            if df_results.loc[ind,'tourney_level'] == 'A':  # ATP 250, 500
               df_1_player_aux_1.loc[0,"qty_matches_ATP_250_500"] = df_1_player_aux_1.loc[0,"qty_matches_ATP_250_500"] + 1 
               df_1_player_aux_1.loc[0,"qty_victories_ATP_250_500"] = df_1_player_aux_1.loc[0,"qty_victories_ATP_250_500"] + 1   
               df_1_player_aux_1.loc[0,"pct_victories_ATP_250_500"] = df_1_player_aux_1.loc[0,"qty_victories_ATP_250_500"] / df_1_player_aux_1.loc[0,"qty_matches_ATP_250_500"] * 100  
               if df_results.loc[ind,'round'] == 'F': # Partida final do torneio
                  df_1_player_aux_1.loc[0,"qty_ATP_250_500_titles"] = df_1_player_aux_1.loc[0,"qty_ATP_250_500_titles"] + 1 
                
            elif df_results.loc[ind,'tourney_level'] == 'F':  # ATP Finals
               df_1_player_aux_1.loc[0,"qty_matches_finals"] = df_1_player_aux_1.loc[0,"qty_matches_finals"] + 1 
               df_1_player_aux_1.loc[0,"qty_victories_finals"] = df_1_player_aux_1.loc[0,"qty_victories_finals"] + 1   
               df_1_player_aux_1.loc[0,"pct_victories_finals"] = df_1_player_aux_1.loc[0,"qty_victories_finals"] / df_1_player_aux_1.loc[0,"qty_matches_finals"] * 100 
               if df_results.loc[ind,'round'] == 'F': # Partida final do torneio
                  df_1_player_aux_1.loc[0,"qty_finals_titles"] = df_1_player_aux_1.loc[0,"qty_finals_titles"] + 1  
                    
            elif df_results.loc[ind,'tourney_level'] == 'G':  # Grand Slam   
               df_1_player_aux_1.loc[0,"qty_matches_grand_slam"] = df_1_player_aux_1.loc[0,"qty_matches_grand_slam"] + 1 
               df_1_player_aux_1.loc[0,"qty_victories_grand_slam"] = df_1_player_aux_1.loc[0,"qty_victories_grand_slam"] + 1   
               df_1_player_aux_1.loc[0,"pct_victories_grand_slam"] = df_1_player_aux_1.loc[0,"qty_victories_grand_slam"] / df_1_player_aux_1.loc[0,"qty_matches_grand_slam"] * 100
               if df_results.loc[ind,'round'] == 'F': # Partida final do torneio
                  df_1_player_aux_1.loc[0,"qty_grand_slam_titles"] = df_1_player_aux_1.loc[0,"qty_grand_slam_titles"] + 1  
                    
            elif df_results.loc[ind,'tourney_level'] == 'M':  # Master 1000
               df_1_player_aux_1.loc[0,"qty_matches_master_1000"] = df_1_player_aux_1.loc[0,"qty_matches_master_1000"] + 1 
               df_1_player_aux_1.loc[0,"qty_victories_master_1000"] = df_1_player_aux_1.loc[0,"qty_victories_master_1000"] + 1   
               df_1_player_aux_1.loc[0,"pct_victories_master_1000"] = df_1_player_aux_1.loc[0,"qty_victories_master_1000"] / df_1_player_aux_1.loc[0,"qty_matches_master_1000"] * 100
               if df_results.loc[ind,'round'] == 'F': # Partida final do torneio
                  df_1_player_aux_1.loc[0,"qty_master_1000_titles"] = df_1_player_aux_1.loc[0,"qty_master_1000_titles"] + 1   

            df_1_player = df_1_player.append(df_1_player_aux_1.iloc[0]) 
            df_1_player_geral = df_1_player_geral.append(df_1_player_aux_1.iloc[0]) 
            del df_1_player_aux_1
            
            # ################################################ #
            #         Gera dados agregados do perdedor         #
            # ################################################ #
                
            df_1_player_aux_1 = df_1_player[df_1_player['player_id'] == df_results.loc[ind,'loser_id']]
                                                                                        
            if not df_1_player_aux_1.empty:  
               df_1_player_aux_1.sort_values(['match_seq'],inplace=True,ascending=[False])   
               df_1_player_aux_1.reset_index(drop=True,inplace=True)
               # df_1_player_aux_1.reset_index(inplace=True)
            
               # print('Loser encontrado *********** :{}'.format(df_results.loc[ind,'loser_id']))
                
               # if ano == 2019: 
               #   print(df_1_player_aux_1) 
               #   print('Tourney : ' + df_results.loc[ind,'tourney_id']) 
               #   print('Torneio : ' + df_1_player_aux_1.loc[0,"tourney_id"])
            
            
               df_1_player_aux_1.loc[0,"tourney_id"] = df_results.loc[ind,'tourney_id']
               df_1_player_aux_1.loc[0,"tourney_date"] = df_results.loc[ind,'tourney_date'] 
               df_1_player_aux_1.loc[0,"match_num"] = df_results.loc[ind,'match_num'] 
               df_1_player_aux_1.loc[0,"round"] = df_results.loc[ind,'round']   

            else:
               # print('Loser não encontrado :{}'.format(df_results.loc[ind,'loser_id']))
            
               df_1_player_aux_1 = pd.DataFrame([[
                                df_results.loc[ind,'loser_id'],
                                df_results.loc[ind,'tourney_id'],  
                                df_results.loc[ind,'tourney_date'],  
                                df_results.loc[ind,'round'],  
                                df_results.loc[ind,'match_num'],
                                0 ,   # match_seq  
                                0 ,   # qty_grass_matches
                                0 ,   # qty_grass_victories
                                0.0 , # pct_grass_victories
                                0 ,   # qty_hard_matches
                                0 ,   # qty_hard_victories
                                0.0 , # pct_hard_victories
                                0 ,   # qty_clay_matches
                                0 ,   # qty_clay_victories
                                0.0 , # pct_clay_victories
                                0 ,   # qty_matches_against_left_handed
                                0 ,   # qty_victories_against_left_handed
                                0.0 , # pct_victories_against_left_handed
                                0 ,   # qty_3sets_matches
                                0 ,   # qty_3sets_victories
                                0.0 , # pct_3sets_victories
                                0 ,   # qty_5sets_matches
                                0 ,   # qty_5sets_victories
                                0.0 , # pct_5sets_victories
                                0   , # minutes
                                0.0 , # avg_minutes
                                0 ,   # qty_ace
                                0 ,   # qty_df
                                0 ,   # qty_svpt
                                0 ,   # qty_1stIn
                                0 ,   # qty_1stWon
                                0 ,   # qty_2ndWon
                                0 ,   # qty_SvGms
                                0 ,   # qty_bpSaved
                                0 ,   # qty_bpFaced
                                0.0 , # avg_ace
                                0.0 , # avg_df
                                0.0 , # avg_svpt
                                0.0 , # avg_1stIn
                                0.0 , # avg_1stWon
                                0.0 , # avg_2ndWon
                                0.0 , # avg_SvGms
                                0.0 , # avg_bpSaved
                                0.0 , # avg_bpFaced
                                0.0 , # height
                                0.0 , # age
                                0   , # rank_points
                                0 ,   # qty_matches_grand_slam
                                0 ,   # qty_victories_grand_slam
                                0.0 , # pct_victories_grand_slam
                                0 ,   # qty_matches_master_1000
                                0 ,   # qty_victories_master_1000
                                0.0 , # pct_victories_master_1000
                                0 ,   # qty_matches_ATP_250_500
                                0 ,   # qty_victories_ATP_250_500
                                0.0 , # pct_victories_ATP_250_500
                                0 ,   # qty_matches_finals
                                0 ,   # qty_victories_finals
                                0.0 , # pct_victories_finals
                                0 ,   # qty_grand_slam_titles
                                0 ,   # qty_master_1000_titles
                                0 ,   # qty_ATP_250_500_titles
                                0     # qty_finals_titles
                                       ]], columns = columns_1 )    
                
            df_1_player_aux_1.loc[0,"match_seq"] = df_1_player_aux_1.loc[0,"match_seq"] + 1
            
            if df_results.loc[ind,'surface'] == 'Clay':
               df_1_player_aux_1.loc[0,"qty_clay_matches"] = df_1_player_aux_1.loc[0,"qty_clay_matches"] + 1
               df_1_player_aux_1.loc[0,"pct_clay_victories"] = df_1_player_aux_1.loc[0,"qty_clay_victories"] / df_1_player_aux_1.loc[0,"qty_clay_matches"] * 100     
            elif df_results.loc[ind,'surface'] == 'Grass':   
                 df_1_player_aux_1.loc[0,"qty_grass_matches"] = df_1_player_aux_1.loc[0,"qty_grass_matches"] + 1
                 df_1_player_aux_1.loc[0,"pct_grass_victories"] = df_1_player_aux_1.loc[0,"qty_grass_victories"] / df_1_player_aux_1.loc[0,"qty_grass_matches"] * 100  
            else:
               df_1_player_aux_1.loc[0,"qty_hard_matches"] = df_1_player_aux_1.loc[0,"qty_hard_matches"] + 1 
               df_1_player_aux_1.loc[0,"pct_hard_victories"] = df_1_player_aux_1.loc[0,"qty_hard_victories"] / df_1_player_aux_1.loc[0,"qty_hard_matches"] * 100  

                
            if df_results.loc[ind,'winner_hand'] == 'L':
               df_1_player_aux_1.loc[0,"qty_matches_against_left_handed"] = df_1_player_aux_1.loc[0,"qty_matches_against_left_handed"] + 1
               df_1_player_aux_1.loc[0,"pct_victories_against_left_handed"] = df_1_player_aux_1.loc[0,"qty_victories_against_left_handed"] / df_1_player_aux_1.loc[0,"qty_matches_against_left_handed"] * 100  
            
            if df_results.loc[ind,'best_of'] == '3':
               df_1_player_aux_1.loc[0,"qty_3sets_matches"] = df_1_player_aux_1.loc[0,"qty_3sets_matches"] + 1
               df_1_player_aux_1.loc[0,"pct_3sets_victories"] = df_1_player_aux_1.loc[0,"qty_3sets_victories"] / df_1_player_aux_1.loc[0,"qty_3sets_matches"] * 100   
            else:
               df_1_player_aux_1.loc[0,"qty_5sets_matches"] = df_1_player_aux_1.loc[0,"qty_5sets_matches"] + 1
               df_1_player_aux_1.loc[0,"pct_5sets_victories"] = df_1_player_aux_1.loc[0,"qty_5sets_victories"] / df_1_player_aux_1.loc[0,"qty_5sets_matches"] * 100   
 
            df_1_player_aux_1.loc[0,"minutes"] = df_1_player_aux_1.loc[0,"minutes"] + int("0" + df_results.loc[ind,'minutes'])
            df_1_player_aux_1.loc[0,"avg_minutes"] = df_1_player_aux_1.loc[0,"minutes"] / df_1_player_aux_1.loc[0,"match_seq"]
    
            df_1_player_aux_1.loc[0,"qty_ace"] = df_1_player_aux_1.loc[0,"qty_ace"] + int("0" + df_results.loc[ind,'l_ace'])
            df_1_player_aux_1.loc[0,"qty_df"] = df_1_player_aux_1.loc[0,"qty_df"] + int("0" + df_results.loc[ind,'l_df'])
            df_1_player_aux_1.loc[0,"qty_svpt"] = df_1_player_aux_1.loc[0,"qty_svpt"] + int("0" + df_results.loc[ind,'l_svpt'])
            df_1_player_aux_1.loc[0,"qty_1stIn"] = df_1_player_aux_1.loc[0,"qty_1stIn"] + int("0" + df_results.loc[ind,'l_1stIn'])
            df_1_player_aux_1.loc[0,"qty_1stWon"] = df_1_player_aux_1.loc[0,"qty_1stWon"] + int("0" + df_results.loc[ind,'l_1stWon'])
            df_1_player_aux_1.loc[0,"qty_2ndWon"] = df_1_player_aux_1.loc[0,"qty_2ndWon"] + int("0" + df_results.loc[ind,'l_2ndWon'])
            df_1_player_aux_1.loc[0,"qty_SvGms"] = df_1_player_aux_1.loc[0,"qty_SvGms"] + int("0" + df_results.loc[ind,'l_SvGms'])
            df_1_player_aux_1.loc[0,"qty_bpSaved"] = df_1_player_aux_1.loc[0,"qty_bpSaved"] + int("0" + df_results.loc[ind,'l_bpSaved'])
            df_1_player_aux_1.loc[0,"qty_bpFaced"] = df_1_player_aux_1.loc[0,"qty_bpFaced"] + int("0" + df_results.loc[ind,'l_bpFaced'])

            df_1_player_aux_1.loc[0,"avg_ace"] = df_1_player_aux_1.loc[0,"qty_ace"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_df"] = df_1_player_aux_1.loc[0,"qty_df"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_svpt"] = df_1_player_aux_1.loc[0,"qty_svpt"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_1stIn"] = df_1_player_aux_1.loc[0,"qty_1stIn"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_1stWon"] = df_1_player_aux_1.loc[0,"qty_1stWon"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_2ndWon"] = df_1_player_aux_1.loc[0,"qty_2ndWon"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_SvGms"] = df_1_player_aux_1.loc[0,"qty_SvGms"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_bpSaved"] = df_1_player_aux_1.loc[0,"qty_bpSaved"] / df_1_player_aux_1.loc[0,"match_seq"]
            df_1_player_aux_1.loc[0,"avg_bpFaced"] = df_1_player_aux_1.loc[0,"qty_bpFaced"] / df_1_player_aux_1.loc[0,"match_seq"]
            
            df_1_player_aux_1.loc[0,"height"] = int("0" + df_results.loc[ind,'loser_ht'])
            df_1_player_aux_1.loc[0,"age"] =  float("0" + df_results.loc[ind,'loser_age'])
            df_1_player_aux_1.loc[0,"rank_points"] = int("0" + df_results.loc[ind,'loser_rank_points'])            
            
            if df_results.loc[ind,'tourney_level'] == 'A':  # ATP 250, 500
               df_1_player_aux_1.loc[0,"qty_matches_ATP_250_500"] = df_1_player_aux_1.loc[0,"qty_matches_ATP_250_500"] + 1 
               df_1_player_aux_1.loc[0,"pct_victories_ATP_250_500"] = df_1_player_aux_1.loc[0,"qty_victories_ATP_250_500"] / df_1_player_aux_1.loc[0,"qty_matches_ATP_250_500"] * 100              
            elif df_results.loc[ind,'tourney_level'] == 'F':  # ATP Finals
               df_1_player_aux_1.loc[0,"qty_matches_finals"] = df_1_player_aux_1.loc[0,"qty_matches_finals"] + 1 
               df_1_player_aux_1.loc[0,"pct_victories_finals"] = df_1_player_aux_1.loc[0,"qty_victories_finals"] / df_1_player_aux_1.loc[0,"qty_matches_finals"] * 100 
            elif df_results.loc[ind,'tourney_level'] == 'G':  # Grand Slam   
               df_1_player_aux_1.loc[0,"qty_matches_grand_slam"] = df_1_player_aux_1.loc[0,"qty_matches_grand_slam"] + 1 
               df_1_player_aux_1.loc[0,"pct_victories_grand_slam"] = df_1_player_aux_1.loc[0,"qty_victories_grand_slam"] / df_1_player_aux_1.loc[0,"qty_matches_grand_slam"] * 100
            elif df_results.loc[ind,'tourney_level'] == 'M':  # Master 1000
               df_1_player_aux_1.loc[0,"qty_matches_master_1000"] = df_1_player_aux_1.loc[0,"qty_matches_master_1000"] + 1 
               df_1_player_aux_1.loc[0,"pct_victories_master_1000"] = df_1_player_aux_1.loc[0,"qty_victories_master_1000"] / df_1_player_aux_1.loc[0,"qty_matches_master_1000"] * 100                
 
            df_1_player = df_1_player.append(df_1_player_aux_1.iloc[0])      
            df_1_player_geral = df_1_player_geral.append(df_1_player_aux_1.iloc[0]) 
            del df_1_player_aux_1
        
            # ################################################################# #
            #         Gera dados agregados do vencedor e perdedor juntos        #
            # ################################################################# #
            
            if df_results.loc[ind,'winner_id'] < df_results.loc[ind,'loser_id']:
               player_1_id = df_results.loc[ind,'winner_id']
               player_2_id = df_results.loc[ind,'loser_id']
            else:
               player_1_id = df_results.loc[ind,'loser_id']
               player_2_id = df_results.loc[ind,'winner_id']                
                
            # df3x = df3[(df3['winner_id'] == '103720') & (df3['match_num'] == 1)]    
                
            df_2_player_aux_1 = df_2_player[(df_2_player['player_1_id'] == player_1_id) & (df_2_player['player_2_id'] == player_2_id)]
                                                                                        
            if not df_2_player_aux_1.empty:                
               df_2_player_aux_1.sort_values(['match_seq'],inplace=True,ascending=[False])   
               df_2_player_aux_1.reset_index(drop=True,inplace=True)
            
               df_2_player_aux_1.loc[0,"tourney_id"] = df_results.loc[ind,'tourney_id']
               df_2_player_aux_1.loc[0,"tourney_date"] = df_results.loc[ind,'tourney_date'] 
               df_2_player_aux_1.loc[0,"match_num"] = df_results.loc[ind,'match_num'] 
               df_2_player_aux_1.loc[0,"round"] = df_results.loc[ind,'round']  
               
            else:
               df_2_player_aux_1 = pd.DataFrame([[
                                player_1_id,
                                player_2_id,
                                df_results.loc[ind,'tourney_id'],  
                                df_results.loc[ind,'tourney_date'],  
                                df_results.loc[ind,'round'],
                                df_results.loc[ind,'match_num'],
                                0 ,   #  match_seq
                                0 ,   #  qty_grass_matches
                                0 ,   #  qty_player_1_grass_victories
                                0.0 , #  pct_player_1_grass_victories
                                0 ,   #  qty_hard_matches
                                0 ,   #  qty_player_1_hard_victories
                                0.0 , #  pct_player_1_hard_victories
                                0 ,   #  qty_clay_matches
                                0 ,   #  qty_player_1_clay_victories
                                0.0 , #  pct_player_1_clay_victories
                                0 ,   #  qty_3sets_matches
                                0 ,   #  qty_player_1_3sets_victories
                                0.0 , #  pct_player_1_3sets_victories
                                0 ,   #  qty_5sets_matches
                                0 ,   #  qty_player_1_5sets_victories
                                0.0 , #  pct_player_1_5sets_victories
                                0 ,   #  qty_grand_slam_matches
                                0 ,   #  qty_player_1_grand_slam_victories
                                0.0 , #  pct_player_1_grand_slam_victories
                                0 ,   #  qty_master_1000_matches
                                0 ,   #  qty_player_1_master_1000_victories
                                0.0 , #  pct_player_1_master_1000_victories
                                0 ,   #  qty_finals_matches
                                0 ,   #  qty_player_1_finals_victories
                                0.0 , #  pct_player_1_finals_victories
                                0 ,   #  qty_ATP_250_500_matches
                                0 ,   #  qty_player_1_ATP_250_500_victories
                                0.0   #  pct_player_1_ATP_250_500_victories
                                       ]], columns = columns_2)    
            
            df_2_player_aux_1.loc[0,"match_seq"] = df_2_player_aux_1.loc[0,"match_seq"] + 1
            
            if df_results.loc[ind,'surface'] == 'Clay':
               df_2_player_aux_1.loc[0,"qty_clay_matches"] = df_2_player_aux_1.loc[0,"qty_clay_matches"] + 1  
               if player_1_id == df_results.loc[ind,'winner_id']: 
                  df_2_player_aux_1.loc[0,"qty_player_1_clay_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_clay_victories"] + 1   
                  df_2_player_aux_1.loc[0,"pct_player_1_clay_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_clay_victories"] / df_2_player_aux_1.loc[0,"qty_clay_matches"] * 100  
            elif df_results.loc[ind,'surface'] == 'Grass':   
               df_2_player_aux_1.loc[0,"qty_grass_matches"] = df_2_player_aux_1.loc[0,"qty_grass_matches"] + 1  
               if player_1_id == df_results.loc[ind,'winner_id']: 
                  df_2_player_aux_1.loc[0,"qty_player_1_grass_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_grass_victories"] + 1   
                  df_2_player_aux_1.loc[0,"pct_player_1_grass_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_grass_victories"] / df_2_player_aux_1.loc[0,"qty_grass_matches"] * 100  
            else:        
               df_2_player_aux_1.loc[0,"qty_hard_matches"] = df_2_player_aux_1.loc[0,"qty_hard_matches"] + 1  
               if player_1_id == df_results.loc[ind,'winner_id']: 
                  df_2_player_aux_1.loc[0,"qty_player_1_hard_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_hard_victories"] + 1   
                  df_2_player_aux_1.loc[0,"pct_player_1_hard_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_hard_victories"] / df_2_player_aux_1.loc[0,"qty_hard_matches"] * 100  

            
            if df_results.loc[ind,'best_of'] == '3':
               df_2_player_aux_1.loc[0,"qty_3sets_matches"] = df_2_player_aux_1.loc[0,"qty_3sets_matches"] + 1  
               if player_1_id == df_results.loc[ind,'winner_id']: 
                  df_2_player_aux_1.loc[0,"qty_player_1_3sets_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_3sets_victories"] + 1   
                  df_2_player_aux_1.loc[0,"pct_player_1_3sets_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_3sets_victories"] / df_2_player_aux_1.loc[0,"qty_3sets_matches"] * 100  
            else:
               df_2_player_aux_1.loc[0,"qty_5sets_matches"] = df_2_player_aux_1.loc[0,"qty_5sets_matches"] + 1  
               if player_1_id == df_results.loc[ind,'winner_id']: 
                  df_2_player_aux_1.loc[0,"qty_player_1_5sets_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_5sets_victories"] + 1   
                  df_2_player_aux_1.loc[0,"pct_player_1_5sets_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_5sets_victories"] / df_2_player_aux_1.loc[0,"qty_5sets_matches"] * 100  

                    
            if df_results.loc[ind,'tourney_level'] == 'A':  # ATP 250, 500
               df_2_player_aux_1.loc[0,"qty_ATP_250_500_matches"] = df_2_player_aux_1.loc[0,"qty_ATP_250_500_matches"] + 1  
               if player_1_id == df_results.loc[ind,'winner_id']: 
                  df_2_player_aux_1.loc[0,"qty_player_1_ATP_250_500_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_ATP_250_500_victories"] + 1   
                  df_2_player_aux_1.loc[0,"pct_player_1_ATP_250_500_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_ATP_250_500_victories"] / df_2_player_aux_1.loc[0,"qty_ATP_250_500_matches"] * 100  
            elif df_results.loc[ind,'tourney_level'] == 'F':  # ATP Finals
               df_2_player_aux_1.loc[0,"qty_finals_matches"] = df_2_player_aux_1.loc[0,"qty_finals_matches"] + 1  
               if player_1_id == df_results.loc[ind,'winner_id']: 
                  df_2_player_aux_1.loc[0,"qty_player_1_finals_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_finals_victories"] + 1   
                  df_2_player_aux_1.loc[0,"pct_player_1_finals_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_finals_victories"] / df_2_player_aux_1.loc[0,"qty_finals_matches"] * 100
            elif df_results.loc[ind,'tourney_level'] == 'G':  # Grand Slam   
               df_2_player_aux_1.loc[0,"qty_grand_slam_matches"] = df_2_player_aux_1.loc[0,"qty_grand_slam_matches"] + 1  
               if player_1_id == df_results.loc[ind,'winner_id']: 
                  df_2_player_aux_1.loc[0,"qty_player_1_grand_slam_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_grand_slam_victories"] + 1   
                  df_2_player_aux_1.loc[0,"pct_player_1_grand_slam_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_grand_slam_victories"] / df_2_player_aux_1.loc[0,"qty_grand_slam_matches"] * 100  
            elif df_results.loc[ind,'tourney_level'] == 'M':  # Master 1000
               df_2_player_aux_1.loc[0,"qty_master_1000_matches"] = df_2_player_aux_1.loc[0,"qty_master_1000_matches"] + 1  
               if player_1_id == df_results.loc[ind,'winner_id']: 
                  df_2_player_aux_1.loc[0,"qty_player_1_master_1000_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_master_1000_victories"] + 1   
                  df_2_player_aux_1.loc[0,"pct_player_1_master_1000_victories"] = df_2_player_aux_1.loc[0,"qty_player_1_master_1000_victories"] / df_2_player_aux_1.loc[0,"qty_master_1000_matches"] * 100  
 
            df_2_player = df_2_player.append(df_2_player_aux_1.iloc[0])      
            df_2_player_geral = df_2_player_geral.append(df_2_player_aux_1.iloc[0]) 

            del df_2_player_aux_1
            
        df_1_player_last = df_1_player.groupby(['player_id'], sort=False).agg({'match_seq':'max'})
        df_1_player =  df_1_player.merge(df_1_player_last, how='inner', right_on=['player_id', 'match_seq'], left_on=['player_id', 'match_seq'])
        # df_1_player.reset_index(inplace=True)
        print(df_1_player)
        df_1_player.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\df1_player_' + str(ano) + '.csv', index = False)
        
        df_2_player_last = df_2_player.groupby(['player_1_id', 'player_2_id'], sort=False).agg({'match_seq':'max'})
        df_2_player =  df_2_player.merge(df_2_player_last, how='inner', right_on=['player_1_id', 'player_2_id', 'match_seq'], left_on=['player_1_id', 'player_2_id', 'match_seq'])
        # df_2_player.reset_index(inplace=True)
        df_2_player.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\df2_player_' + str(ano) + '.csv', index = False)
        
    now = datetime.now()              
    print(now.strftime("%H:%M:%S"))         
    df_1_player_geral.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\df1_player_geral' + '.csv', index = False)
    df_2_player_geral.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\df2_player_geral' + '.csv', index = False)
    print("OK")                

In [23]:
def select_winner (row):
    if row['winner_id'] < row['loser_id'] :
      return 1
    else:
      return 2

In [24]:
def select_winner_player_1_2 (row):
    if row['winner_id'] < row['loser_id'] :
       row['winner'] = 1
       row['player_1_id'] = row['winner_id']
       row['player_2_id'] = row['loser_id']
    else:
       row['winner'] = 2
       row['player_1_id'] = row['loser_id']
       row['player_2_id'] = row['winner_id']
    
    return row

In [25]:
def select_player_1 (row):
    if row['winner_id'] < row['loser_id'] :
      return row['winner_id']
    else:
      return row['loser_id']

In [26]:
def select_player_2 (row):
    if row['winner_id'] < row['loser_id'] :
      return row['loser_id']
    else:
      return row['winner_id']

In [27]:
def gera_ATP_ODDS_final():

    # Obtem os dados de merge de resultados da ATP e odds de apostas
    # --------------------------------------------------------------
    
    df_ATP_ODDS = pd.read_excel (r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\Saved_2\merged_ATP_ODDS.xlsx')
    df_ATP_ODDS = df_ATP_ODDS.filter(['tourney_id','tourney_date','match_num', 'round','winner_id','loser_id','surface','tourney_level','best_of','AvgW','AvgL'])
    
    #      Adiciona a coluna winner para indicar que foi o vencedor : jogador 1 ou 2
    #      -------------------------------------------------------------------------
    
    # df_ATP_ODDS['winner'] = df_ATP_ODDS.apply(lambda row: select_winner (row), axis=1)
    
    #      Adiciona as colunas winner,  player_1 e player_2 para poder fazer o merge com os dados agregados do par de jogadores 
    #      --------------------------------------------------------------------------------------------------------------------
        
    # df_ATP_ODDS[['player_1', 'player_2']] = df_ATP_ODDS.apply(lambda row: select_player_1_2 (row), axis=1)
    # df_ATP_ODDS['player_1'] = df_ATP_ODDS.apply(lambda row: select_player_1 (row), axis=1)
    # df_ATP_ODDS['player_2'] = df_ATP_ODDS.apply(lambda row: select_player_2 (row), axis=1)
    
    df_ATP_ODDS=df_ATP_ODDS.apply(select_winner_player_1_2,axis=1)
    
    #      Elimina linhas com ODDS nulos
    #      -----------------------------
    
    df_ATP_ODDS = df_ATP_ODDS[df_ATP_ODDS['AvgW'].notnull()]
    df_ATP_ODDS = df_ATP_ODDS[df_ATP_ODDS['AvgL'].notnull()]
    
    print(df_ATP_ODDS)
    
    df_ATP_ODDS.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\Saved_2\merged_ATP_ODDS_final.csv', index = False)


In [119]:
def gera_bases_finais():
    
    # Obtem os dados de merge de resultados da ATP e odds de apostas
    # --------------------------------------------------------------
    
    df_ATP_ODDS = pd.read_csv('C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\Saved_2\merged_ATP_ODDS_final.csv', sep=",")
    
    # Obtem os dados agregados de cada jogador
    # ----------------------------------------
    
    df1_player = pd.read_csv('C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\Saved_2\df1_player_geral.csv', sep=";")
    df1_player_aux = df1_player
    
    #      Altera o número de sequência dos jogos para que os dados agregados fiquem associados às partidas seguintes
    #      ----------------------------------------------------------------------------------------------------------
    
    df1_player_aux.match_seq = df1_player.match_seq + 1
    
    #      Obtem as colunas relevantes
    #      ---------------------------
    
    df1_player_aux.merge(df1_player, how='left', left_on=['player_id', 'match_seq'], right_on=['player_id', 'match_seq'])
    df1_player_aux = df1_player_aux.filter(['player_id', 'tourney_id', 'tourney_date', 'match_num', 'round','pct_grass_victories','pct_hard_victories','pct_clay_victories','pct_3sets_victories','pct_5sets_victories','avg_ace','avg_df','avg_svpt','avg_1stIn','avg_1stWon','avg_2ndWon','avg_SvGms','avg_bpSaved','avg_bpFaced','age','rank_points','pct_victories_grand_slam','pct_victories_master_1000','pct_victories_ATP_250_500','pct_victories_finals','qty_grand_slam_titles','qty_master_1000_titles','qty_ATP_250_500_titles','qty_finals_titles'])
    
    df1_player_aux.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\df1_player_aux_XX.csv', index = False, sep=';')
    
    # Obtem os dados agregados de cada par de jogadores
    # -------------------------------------------------
    
    df2_player = pd.read_csv('C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\Saved_2\df2_player_geral.csv', sep=";")    
    df2_player_aux = df2_player
    
    #      Altera o número de sequência dos jogos para que os dados agregados fiquem associados às partidas seguintes
    #      ----------------------------------------------------------------------------------------------------------
    
    df2_player_aux.match_seq = df2_player.match_seq + 1
    
    #      Obtem as colunas relevantes
    #      ---------------------------
    
    df2_player_aux.merge(df2_player, how='left', left_on=['player_1_id', 'player_2_id', 'match_seq'], right_on=['player_1_id', 'player_2_id', 'match_seq'])
    df2_player_aux = df2_player_aux.filter(['player_1_id', 'player_2_id', 'tourney_id', 'tourney_date', 'match_num', 'round','pct_player_1_grass_victories','pct_player_1_hard_victories','pct_player_1_clay_victories','pct_player_1_3sets_victories','pct_player_1_5sets_victories','pct_player_1_grand_slam_victories','pct_player_1_master_1000_victories','pct_player_1_finals_victories','pct_player_1_ATP_250_500_victories','qty_grass_matches','qty_hard_matches','qty_clay_matches','qty_3sets_matches','qty_5sets_matches','qty_grand_slam_matches','qty_master_1000_matches','qty_finals_matches','qty_ATP_250_500_matches'
                                           ])
    
    df2_player_aux.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\df2_player_aux_XX.csv', index = False, sep=';')
    
    # Merge com dados agregados do winner
    # -----------------------------------
    
    df_merge_all =  df_ATP_ODDS.merge(df1_player_aux, how='left', left_on=['winner_id', 'tourney_id', 'tourney_date', 'match_num', 'round'], right_on=['player_id', 'tourney_id', 'tourney_date', 'match_num', 'round'])
    
    # Merge com dados agregados do loser
    # -----------------------------------
    
    df_merge_all =  df_merge_all.merge(df1_player_aux, how='left', left_on=['loser_id', 'tourney_id', 'tourney_date', 'match_num', 'round'], right_on=['player_id', 'tourney_id', 'tourney_date', 'match_num', 'round'])

    # Merge com dados agregados do par de jogadores
    # ---------------------------------------------
    
    df_merge_all =  df_merge_all.merge(df2_player_aux, how='left', left_on=['player_1_id', 'player_2_id', 'tourney_id', 'tourney_date', 'match_num', 'round'], right_on=['player_1_id', 'player_2_id', 'tourney_id', 'tourney_date', 'match_num', 'round'])
    
    df_merge_all.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\merge_all_XX.csv', index = False , sep=';')
    
    # Gera base final completa
    # ------------------------
    
    columns = {
             'tourney_id':'',
             'tourney_date':'',
             'match_num':int,
             'round':'',
             'surface':'',
             'tourney_level':'',
             'best_of':int,
             'AvgW':float,
             'AvgL':float,

             'player_id_1':int,
             'pct_surface_victories_1':float,
             'pct_best_of_victories_1':float,
             'avg_ace_1':float,
             'avg_df_1':float,
             'avg_svpt_1':float,
             'avg_1stIn_1':float,
             'avg_1stWon_1':float,
             'avg_2ndWon_1':float,
             'avg_SvGms_1':float,
             'avg_bpSaved_1':float,
             'avg_bpFaced_1':float,
             'age_1':int,
             'rank_points_1':int,
             'pct_victories_grand_slam_1':float,
             'pct_victories_master_1000_1':float,
             'pct_victories_ATP_250_500_1':float,
             'pct_victories_finals_1':float,
             'qty_grand_slam_titles_1':int,
             'qty_master_1000_titles_1':int,
             'qty_ATP_250_500_titles_1':int,
             'qty_finals_titles_1':int,

             'player_id_2':int,
             'pct_surface_victories_2':float,
             'pct_best_of_victories_2':float,
             'avg_ace_2':float,
             'avg_df_2':float,
             'avg_svpt_2':float,
             'avg_1stIn_2':float,
             'avg_1stWon_2':float,
             'avg_2ndWon_2':float,
             'avg_SvGms_2':float,
             'avg_bpSaved_2':float,
             'avg_bpFaced_2':float,
             'age_2':int,
             'rank_points_2':int,
             'pct_victories_grand_slam_2':float,
             'pct_victories_master_1000_2':float,
             'pct_victories_ATP_250_500_2':float,
             'pct_victories_finals_2':float,
             'qty_grand_slam_titles_2':int,
             'qty_master_1000_titles_2':int,
             'qty_ATP_250_500_titles_2':int,
             'qty_finals_titles_2':int,

             'pct_player_1_surface_victories':float,
             'pct_player_1_best_of_victories':float,
             'pct_player_1_grand_slam_victories':float,
             'pct_player_1_master_1000_victories':float,
             'pct_player_1_finals_victories':float,
             'pct_player_1_ATP_250_500_victories':float,

             'pct_player_2_surface_victories':float,
             'pct_player_2_best_of_victories':float,
             'pct_player_2_grand_slam_victories':float,
             'pct_player_2_master_1000_victories':float,
             'pct_player_2_finals_victories':float,
             'pct_player_2_ATP_250_500_victories':float,

             'winner':int
             }

    contador = 0
    
    df_base_completa = pd.DataFrame(columns, index=[])    
    
    for ind in df_merge_all.index:
        
        contador = contador + 1

        if (contador % 100) == 1:
           now = datetime.now()
           print(str(contador) + '-' + now.strftime("%H:%M:%S")) 
        
        # if contador == 302:
        #    break 
            
        # print('250: ' + df_merge_all.loc[ind,'pct_player_1_ATP_250_500_victories'])    
        
        df_base_completa_aux = pd.DataFrame([[
                               df_merge_all.loc[ind,'tourney_id'],
                               df_merge_all.loc[ind,'tourney_date'],
                               df_merge_all.loc[ind,'match_num'],
                               df_merge_all.loc[ind,'round'],
                               df_merge_all.loc[ind,'surface'],
                               df_merge_all.loc[ind,'tourney_level'],
                               df_merge_all.loc[ind,'best_of'],
                               df_merge_all.loc[ind,'AvgW'],
                               df_merge_all.loc[ind,'AvgL'],

                               0,    # player_id_1
                               0,  # pct_surface_victories_1
                               0,  # pct_best_of_victories_1
                               0,  # avg_ace_1
                               0,  # avg_df_1
                               0,  # avg_svpt_1
                               0,  # avg_1stIn_1
                               0,  # avg_1stWon_1
                               0,  # avg_2ndWon_1
                               0,  # avg_SvGms_1
                               0,  # avg_bpSaved_1
                               0,  # avg_bpFaced_1
                               0,    # age_1
                               0,    # rank_points_1
                               0,  # pct_victories_grand_slam_1
                               0,  # pct_victories_master_1000_1
                               0,  # pct_victories_ATP_250_500_1
                               0,  # pct_victories_finals_1
                               0,    # qty_grand_slam_titles_1
                               0,    # qty_master_1000_titles_1
                               0,    # qty_ATP_250_500_titles_1
                               0,    # qty_finals_titles_1

                               0,    # player_id_2
                               0,  # pct_surface_victories_2
                               0,  # pct_best_of_victories_2
                               0,  # avg_ace_2
                               0,  # avg_df_2
                               0,  # avg_svpt_2
                               0,  # avg_1stIn_2
                               0,  # avg_1stWon_2
                               0,  # avg_2ndWon_2
                               0,  # avg_SvGms_2
                               0,  # avg_bpSaved_2
                               0,  # avg_bpFaced_2
                               0,    # age_2
                               0,    # rank_points_2
                               0,  # pct_victories_grand_slam_2
                               0,  # pct_victories_master_1000_2
                               0,  # pct_victories_ATP_250_500_2
                               0,  # pct_victories_finals_2
                               0,    # qty_grand_slam_titles_2
                               0,    # qty_master_1000_titles_2
                               0,    # qty_ATP_250_500_titles_2
                               0,    # qty_finals_titles_2

                               0,    # pct_player_1_surface_victories
                               0,    # pct_player_1_best_of_victories
                               0,    # pct_player_1_grand_slam_victories
                               0,    # pct_player_1_master_1000_victories
                               0,    # pct_player_1_finals_victories
                               0,    # pct_player_1_ATP_250_500_victories

                               0,    # pct_player_2_surface_victories
                               0,    # pct_player_2_best_of_victories
                               0,    # pct_player_2_grand_slam_victories
                               0,    # pct_player_2_master_1000_victories
                               0,    # pct_player_2_finals_victories
                               0,    # pct_player_2_ATP_250_500_victories

                               df_merge_all.loc[ind,'winner'] 
                               
                                       ]], columns = columns)  
        
        if df_merge_all.loc[ind,'winner'] == 1:
            
           # Player_1
           # --------
            
           df_base_completa_aux.loc[0,"player_id_1"] = df_merge_all.loc[ind,"player_id_x"] 
        
           if df_merge_all.loc[ind,'surface'] == 'hard':
              df_base_completa_aux.loc[0,"pct_surface_victories_1"] = df_merge_all.loc[ind,"pct_hard_victories_x"] 
           elif df_merge_all.loc[ind,'surface'] == 'clay': 
              df_base_completa_aux.loc[0,"pct_surface_victories_1"] = df_merge_all.loc[ind,"pct_clay_victories_x"] 
           else:     
              df_base_completa_aux.loc[0,"pct_surface_victories_1"] = df_merge_all.loc[ind,"pct_grass_victories_x"] 
                
           if df_merge_all.loc[ind,'best_of'] == 3:     
              df_base_completa_aux.loc[0,"pct_best_of_victories_1"] = df_merge_all.loc[ind,"pct_3sets_victories_x"] 
           else:
              df_base_completa_aux.loc[0,"pct_best_of_victories_1"] = df_merge_all.loc[ind,"pct_5sets_victories_x"] 
        
           df_base_completa_aux.loc[0,"avg_ace_1"] = df_merge_all.loc[ind,"avg_ace_x"] 
           df_base_completa_aux.loc[0,"avg_df_1"] = df_merge_all.loc[ind,"avg_df_x"] 
           df_base_completa_aux.loc[0,"avg_svpt_1"] = df_merge_all.loc[ind,"avg_svpt_x"] 
           df_base_completa_aux.loc[0,"avg_1stIn_1"] = df_merge_all.loc[ind,"avg_1stIn_x"] 
           df_base_completa_aux.loc[0,"avg_1stWon_1"] = df_merge_all.loc[ind,"avg_1stWon_x"] 
           df_base_completa_aux.loc[0,"avg_2ndWon_1"] = df_merge_all.loc[ind,"avg_2ndWon_x"] 
           df_base_completa_aux.loc[0,"avg_SvGms_1"] = df_merge_all.loc[ind,"avg_SvGms_x"] 
           df_base_completa_aux.loc[0,"avg_bpSaved_1"] = df_merge_all.loc[ind,"avg_bpSaved_x"] 
           df_base_completa_aux.loc[0,"avg_bpFaced_1"] = df_merge_all.loc[ind,"avg_bpFaced_x"] 
           df_base_completa_aux.loc[0,"age_1"] = df_merge_all.loc[ind,"age_x"] 
           df_base_completa_aux.loc[0,"rank_points_1"] = df_merge_all.loc[ind,"rank_points_x"] 
           df_base_completa_aux.loc[0,"pct_victories_grand_slam_1"] = df_merge_all.loc[ind,"pct_victories_grand_slam_x"] 
           df_base_completa_aux.loc[0,"pct_victories_master_1000_1"] = df_merge_all.loc[ind,"pct_victories_master_1000_x"] 
           df_base_completa_aux.loc[0,"pct_victories_ATP_250_500_1"] = df_merge_all.loc[ind,"pct_victories_ATP_250_500_x"] 
           df_base_completa_aux.loc[0,"pct_victories_finals_1"] = df_merge_all.loc[ind,"pct_victories_finals_x"] 
           df_base_completa_aux.loc[0,"qty_grand_slam_titles_1"] = df_merge_all.loc[ind,"qty_grand_slam_titles_x"] 
           df_base_completa_aux.loc[0,"qty_master_1000_titles_1"] = df_merge_all.loc[ind,"qty_master_1000_titles_x"] 
           df_base_completa_aux.loc[0,"qty_ATP_250_500_titles_1"] = df_merge_all.loc[ind,"qty_ATP_250_500_titles_x"] 
           df_base_completa_aux.loc[0,"qty_finals_titles_1"] = df_merge_all.loc[ind,"qty_finals_titles_x"] 

           # Player_2
           # --------
    
           df_base_completa_aux.loc[0,"player_id_2"] = df_merge_all.loc[ind,"player_id_y"] 
        
           if df_merge_all.loc[ind,'surface'] == 'hard':
              df_base_completa_aux.loc[0,"pct_surface_victories_2"] = df_merge_all.loc[ind,"pct_hard_victories_y"] 
           elif df_merge_all.loc[ind,'surface'] == 'clay': 
              df_base_completa_aux.loc[0,"pct_surface_victories_2"] = df_merge_all.loc[ind,"pct_clay_victories_y"] 
           else:     
              df_base_completa_aux.loc[0,"pct_surface_victories_2"] = df_merge_all.loc[ind,"pct_grass_victories_y"] 
                
           if df_merge_all.loc[ind,'best_of'] == 3:     
              df_base_completa_aux.loc[0,"pct_best_of_victories_2"] = df_merge_all.loc[ind,"pct_3sets_victories_y"] 
           else:
              df_base_completa_aux.loc[0,"pct_best_of_victories_2"] = df_merge_all.loc[ind,"pct_5sets_victories_y"]     
        
           df_base_completa_aux.loc[0,"avg_ace_2"] = df_merge_all.loc[ind,"avg_ace_y"] 
           df_base_completa_aux.loc[0,"avg_df_2"] = df_merge_all.loc[ind,"avg_df_y"] 
           df_base_completa_aux.loc[0,"avg_svpt_2"] = df_merge_all.loc[ind,"avg_svpt_y"] 
           df_base_completa_aux.loc[0,"avg_1stIn_2"] = df_merge_all.loc[ind,"avg_1stIn_y"] 
           df_base_completa_aux.loc[0,"avg_1stWon_2"] = df_merge_all.loc[ind,"avg_1stWon_y"] 
           df_base_completa_aux.loc[0,"avg_2ndWon_2"] = df_merge_all.loc[ind,"avg_2ndWon_y"] 
           df_base_completa_aux.loc[0,"avg_SvGms_2"] = df_merge_all.loc[ind,"avg_SvGms_y"] 
           df_base_completa_aux.loc[0,"avg_bpSaved_2"] = df_merge_all.loc[ind,"avg_bpSaved_y"] 
           df_base_completa_aux.loc[0,"avg_bpFaced_2"] = df_merge_all.loc[ind,"avg_bpFaced_y"] 
           df_base_completa_aux.loc[0,"age_2"] = df_merge_all.loc[ind,"age_y"] 
           df_base_completa_aux.loc[0,"rank_points_2"] = df_merge_all.loc[ind,"rank_points_y"] 
           df_base_completa_aux.loc[0,"pct_victories_grand_slam_2"] = df_merge_all.loc[ind,"pct_victories_grand_slam_y"] 
           df_base_completa_aux.loc[0,"pct_victories_master_1000_2"] = df_merge_all.loc[ind,"pct_victories_master_1000_y"] 
           df_base_completa_aux.loc[0,"pct_victories_ATP_250_500_2"] = df_merge_all.loc[ind,"pct_victories_ATP_250_500_y"] 
           df_base_completa_aux.loc[0,"pct_victories_finals_2"] = df_merge_all.loc[ind,"pct_victories_finals_y"] 
           df_base_completa_aux.loc[0,"qty_grand_slam_titles_2"] = df_merge_all.loc[ind,"qty_grand_slam_titles_y"] 
           df_base_completa_aux.loc[0,"qty_master_1000_titles_2"] = df_merge_all.loc[ind,"qty_master_1000_titles_y"] 
           df_base_completa_aux.loc[0,"qty_ATP_250_500_titles_2"] = df_merge_all.loc[ind,"qty_ATP_250_500_titles_y"] 
           df_base_completa_aux.loc[0,"qty_finals_titles_2"] = df_merge_all.loc[ind,"qty_finals_titles_y"] 
    
        else:
            
           # Player_1
           # --------
            
           df_base_completa_aux.loc[0,"player_id_1"] = df_merge_all.loc[ind,"player_id_y"] 
        
           if df_merge_all.loc[ind,'surface'] == 'hard':
              df_base_completa_aux.loc[0,"pct_surface_victories_1"] = df_merge_all.loc[ind,"pct_hard_victories_y"] 
           elif df_merge_all.loc[ind,'surface'] == 'clay': 
              df_base_completa_aux.loc[0,"pct_surface_victories_1"] = df_merge_all.loc[ind,"pct_clay_victories_y"] 
           else:     
              df_base_completa_aux.loc[0,"pct_surface_victories_1"] = df_merge_all.loc[ind,"pct_grass_victories_y"] 
                
           if df_merge_all.loc[ind,'best_of'] == 3:     
              df_base_completa_aux.loc[0,"pct_best_of_victories_1"] = df_merge_all.loc[ind,"pct_3sets_victories_y"] 
           else:
              df_base_completa_aux.loc[0,"pct_best_of_victories_1"] = df_merge_all.loc[ind,"pct_5sets_victories_y"]   
        
           df_base_completa_aux.loc[0,"avg_ace_1"] = df_merge_all.loc[ind,"avg_ace_y"] 
           df_base_completa_aux.loc[0,"avg_df_1"] = df_merge_all.loc[ind,"avg_df_y"] 
           df_base_completa_aux.loc[0,"avg_svpt_1"] = df_merge_all.loc[ind,"avg_svpt_y"] 
           df_base_completa_aux.loc[0,"avg_1stIn_1"] = df_merge_all.loc[ind,"avg_1stIn_y"] 
           df_base_completa_aux.loc[0,"avg_1stWon_1"] = df_merge_all.loc[ind,"avg_1stWon_y"] 
           df_base_completa_aux.loc[0,"avg_2ndWon_1"] = df_merge_all.loc[ind,"avg_2ndWon_y"] 
           df_base_completa_aux.loc[0,"avg_SvGms_1"] = df_merge_all.loc[ind,"avg_SvGms_y"] 
           df_base_completa_aux.loc[0,"avg_bpSaved_1"] = df_merge_all.loc[ind,"avg_bpSaved_y"] 
           df_base_completa_aux.loc[0,"avg_bpFaced_1"] = df_merge_all.loc[ind,"avg_bpFaced_y"] 
           df_base_completa_aux.loc[0,"age_1"] = df_merge_all.loc[ind,"age_y"] 
           df_base_completa_aux.loc[0,"rank_points_1"] = df_merge_all.loc[ind,"rank_points_y"] 
           df_base_completa_aux.loc[0,"pct_victories_grand_slam_1"] = df_merge_all.loc[ind,"pct_victories_grand_slam_y"] 
           df_base_completa_aux.loc[0,"pct_victories_master_1000_1"] = df_merge_all.loc[ind,"pct_victories_master_1000_y"] 
           df_base_completa_aux.loc[0,"pct_victories_ATP_250_500_1"] = df_merge_all.loc[ind,"pct_victories_ATP_250_500_y"] 
           df_base_completa_aux.loc[0,"pct_victories_finals_1"] = df_merge_all.loc[ind,"pct_victories_finals_y"] 
           df_base_completa_aux.loc[0,"qty_grand_slam_titles_1"] = df_merge_all.loc[ind,"qty_grand_slam_titles_y"] 
           df_base_completa_aux.loc[0,"qty_master_1000_titles_1"] = df_merge_all.loc[ind,"qty_master_1000_titles_y"] 
           df_base_completa_aux.loc[0,"qty_ATP_250_500_titles_1"] = df_merge_all.loc[ind,"qty_ATP_250_500_titles_y"] 
           df_base_completa_aux.loc[0,"qty_finals_titles_1"] = df_merge_all.loc[ind,"qty_finals_titles_y"] 
            
           # Player_2
           # -------- 
            
           df_base_completa_aux.loc[0,"player_id_2"] = df_merge_all.loc[ind,"player_id_x"] 
        
           if df_merge_all.loc[ind,'surface'] == 'hard':
              df_base_completa_aux.loc[0,"pct_surface_victories_2"] = df_merge_all.loc[ind,"pct_hard_victories_x"] 
           elif df_merge_all.loc[ind,'surface'] == 'clay': 
              df_base_completa_aux.loc[0,"pct_surface_victories_2"] = df_merge_all.loc[ind,"pct_clay_victories_x"] 
           else:     
              df_base_completa_aux.loc[0,"pct_surface_victories_2"] = df_merge_all.loc[ind,"pct_grass_victories_x"] 
                
           if df_merge_all.loc[ind,'best_of'] == 3:     
              df_base_completa_aux.loc[0,"pct_best_of_victories_2"] = df_merge_all.loc[ind,"pct_3sets_victories_x"] 
           else:
              df_base_completa_aux.loc[0,"pct_best_of_victories_2"] = df_merge_all.loc[ind,"pct_5sets_victories_x"]     
        
           df_base_completa_aux.loc[0,"avg_ace_2"] = df_merge_all.loc[ind,"avg_ace_x"] 
           df_base_completa_aux.loc[0,"avg_df_2"] = df_merge_all.loc[ind,"avg_df_x"] 
           df_base_completa_aux.loc[0,"avg_svpt_2"] = df_merge_all.loc[ind,"avg_svpt_x"] 
           df_base_completa_aux.loc[0,"avg_1stIn_2"] = df_merge_all.loc[ind,"avg_1stIn_x"] 
           df_base_completa_aux.loc[0,"avg_1stWon_2"] = df_merge_all.loc[ind,"avg_1stWon_x"] 
           df_base_completa_aux.loc[0,"avg_2ndWon_2"] = df_merge_all.loc[ind,"avg_2ndWon_x"] 
           df_base_completa_aux.loc[0,"avg_SvGms_2"] = df_merge_all.loc[ind,"avg_SvGms_x"] 
           df_base_completa_aux.loc[0,"avg_bpSaved_2"] = df_merge_all.loc[ind,"avg_bpSaved_x"] 
           df_base_completa_aux.loc[0,"avg_bpFaced_2"] = df_merge_all.loc[ind,"avg_bpFaced_x"] 
           df_base_completa_aux.loc[0,"age_2"] = df_merge_all.loc[ind,"age_x"] 
           df_base_completa_aux.loc[0,"rank_points_2"] = df_merge_all.loc[ind,"rank_points_x"] 
           df_base_completa_aux.loc[0,"pct_victories_grand_slam_2"] = df_merge_all.loc[ind,"pct_victories_grand_slam_x"] 
           df_base_completa_aux.loc[0,"pct_victories_master_1000_2"] = df_merge_all.loc[ind,"pct_victories_master_1000_x"] 
           df_base_completa_aux.loc[0,"pct_victories_ATP_250_500_2"] = df_merge_all.loc[ind,"pct_victories_ATP_250_500_x"] 
           df_base_completa_aux.loc[0,"pct_victories_finals_2"] = df_merge_all.loc[ind,"pct_victories_finals_x"] 
           df_base_completa_aux.loc[0,"qty_grand_slam_titles_2"] = df_merge_all.loc[ind,"qty_grand_slam_titles_x"] 
           df_base_completa_aux.loc[0,"qty_master_1000_titles_2"] = df_merge_all.loc[ind,"qty_master_1000_titles_x"] 
           df_base_completa_aux.loc[0,"qty_ATP_250_500_titles_2"] = df_merge_all.loc[ind,"qty_ATP_250_500_titles_x"] 
           df_base_completa_aux.loc[0,"qty_finals_titles_2"] = df_merge_all.loc[ind,"qty_finals_titles_x"] 
            
           if df_merge_all.loc[ind,'surface'] == 'hard':
              if df_merge_all.loc[ind,'qty_hard_matches'] > 0:
                 df_base_completa_aux.loc[0,"pct_player_1_surface_victories"] = df_merge_all.loc[ind,"pct_player_1_grass_victories"] 
                 df_base_completa_aux.loc[0,"pct_player_2_surface_victories"] = 100 - float(df_merge_all.loc[ind,"pct_player_1_grass_victories"].replace(",",".")) 
              else:
                 df_base_completa_aux.loc[0,"pct_player_1_surface_victories"] = 0
                 df_base_completa_aux.loc[0,"pct_player_2_surface_victories"] = 0                            
                                            
           elif df_merge_all.loc[ind,'surface'] == 'clay': 
              if df_merge_all.loc[ind,'qty_clay_matches'] > 0:
                 df_base_completa_aux.loc[0,"pct_player_1_surface_victories"] = df_merge_all.loc[ind,"pct_player_1_clay_victories"] 
                 df_base_completa_aux.loc[0,"pct_player_2_surface_victories"] = 100 - float(df_merge_all.loc[ind,"pct_player_1_clay_victories"].replace(",","."))  
              else:
                 df_base_completa_aux.loc[0,"pct_player_1_surface_victories"] = 0
                 df_base_completa_aux.loc[0,"pct_player_2_surface_victories"] = 0                                                  
           else:      
              if df_merge_all.loc[ind,'qty_grass_matches'] > 0:
                 df_base_completa_aux.loc[0,"pct_player_1_surface_victories"] = df_merge_all.loc[ind,"pct_player_1_grass_victories"] 
                 df_base_completa_aux.loc[0,"pct_player_2_surface_victories"] = 100 - float(df_merge_all.loc[ind,"pct_player_1_grass_victories"].replace(",","."))  
              else:
                 df_base_completa_aux.loc[0,"pct_player_1_surface_victories"] = 0
                 df_base_completa_aux.loc[0,"pct_player_2_surface_victories"] = 0                                                  

                
           if df_merge_all.loc[ind,'best_of'] == 3:     
              if df_merge_all.loc[ind,'qty_3sets_matches'] > 0:       
                 df_base_completa_aux.loc[0,"pct_player_1_best_of_victories"] = df_merge_all.loc[ind,"pct_player_1_3sets_victories"] 
                 df_base_completa_aux.loc[0,"pct_player_2_best_of_victories"] = 100 - float(df_merge_all.loc[ind,"pct_player_1_3sets_victories"].replace(",","."))  
              else:
                 df_base_completa_aux.loc[0,"pct_player_1_best_of_victories"] = 0
                 df_base_completa_aux.loc[0,"pct_player_2_best_of_victories"] = 0                                                        
           else:
              if df_merge_all.loc[ind,'qty_5sets_matches'] > 0:       
                 df_base_completa_aux.loc[0,"pct_player_1_best_of_victories"] = df_merge_all.loc[ind,"pct_player_1_5sets_victories"] 
                 df_base_completa_aux.loc[0,"pct_player_2_best_of_victories"] = 100 - float(df_merge_all.loc[ind,"pct_player_1_5sets_victories"].replace(",","."))  
              else:
                 df_base_completa_aux.loc[0,"pct_player_1_best_of_victories"] = 0
                 df_base_completa_aux.loc[0,"pct_player_2_best_of_victories"] = 0                                             
   
                                            
           if df_merge_all.loc[ind,'qty_grand_slam_matches'] > 0:   
              df_base_completa_aux.loc[0,"pct_player_1_grand_slam_victories"] = df_merge_all.loc[ind,"pct_player_1_grand_slam_victories"] 
              df_base_completa_aux.loc[0,"pct_player_2_grand_slam_victories"] = 100 - float(df_merge_all.loc[ind,"pct_player_1_grand_slam_victories"].replace(",","."))  
           else:
              df_base_completa_aux.loc[0,"pct_player_1_grand_slam_victories"] = 0
              df_base_completa_aux.loc[0,"pct_player_2_grand_slam_victories"] = 0       
                                            
           if df_merge_all.loc[ind,'qty_master_1000_matches'] > 0:   
              df_base_completa_aux.loc[0,"pct_player_1_master_1000_victories"] = df_merge_all.loc[ind,"pct_player_1_master_1000_victories"] 
              df_base_completa_aux.loc[0,"pct_player_2_master_1000_victories"] = 100 - float(df_merge_all.loc[ind,"pct_player_1_master_1000_victories"].replace(",","."))  
           else:
              df_base_completa_aux.loc[0,"pct_player_1_master_1000_victories"] = 0
              df_base_completa_aux.loc[0,"pct_player_2_master_1000_victories"] = 0      
                                            
           if df_merge_all.loc[ind,'qty_finals_matches'] > 0:   
              df_base_completa_aux.loc[0,"pct_player_1_finals_victories"] = df_merge_all.loc[ind,"pct_player_1_finals_victories"] 
              df_base_completa_aux.loc[0,"pct_player_2_finals_victories"] = 100 - float(df_merge_all.loc[ind,"pct_player_1_finals_victories"].replace(",","."))  
           else:
              df_base_completa_aux.loc[0,"pct_player_1_finals_victories"] = 0
              df_base_completa_aux.loc[0,"pct_player_2_finals_victories"] = 0      
                                            
           if df_merge_all.loc[ind,'qty_ATP_250_500_matches'] > 0:   
              df_base_completa_aux.loc[0,"pct_player_1_ATP_250_500_victories"] = df_merge_all.loc[ind,"pct_player_1_ATP_250_500_victories"] 
              df_base_completa_aux.loc[0,"pct_player_2_ATP_250_500_victories"] = 100 - float(df_merge_all.loc[ind,"pct_player_1_ATP_250_500_victories"].replace(",","."))  
           else:
              df_base_completa_aux.loc[0,"pct_player_1_ATP_250_500_victories"] = 0
              df_base_completa_aux.loc[0,"pct_player_2_ATP_250_500_victories"] = 0                                    
                                 
 
        df_base_completa = df_base_completa.append(df_base_completa_aux.iloc[0])   
        del df_base_completa_aux
            
    df_base_completa.to_csv(r'C:\Giba\Dados\_A_Giba\Documentos\Adm\PUC_MG\Python\base_completa_XX.csv', index = False , sep=';')
    df_base_completa.to_excel("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\base_completa_XX.xlsx")
    
    print('OK')

## Execuções

In [68]:
gera_ATP_ODDS_final()

      tourney_id  tourney_date  match_num round  winner_id  loser_id surface  \
3053    2001-339      20010101          1   R32     103720    101885    Hard   
3054    2001-451      20010101          1   R32     102338    101532    Hard   
3055    2001-891      20010101          1   R32     103758    102796    Hard   
3057    2001-451      20010101          2   R32     102966    102106    Hard   
3058    2001-891      20010101          2   R32     103709    102286    Hard   
...          ...           ...        ...   ...        ...       ...     ...   
55609  2019-0605      20191111        296    RR     104745    126774    Hard   
55610  2019-0605      20191111        297    RR     104745    106421    Hard   
55611  2019-0605      20191111        298    SF     106233    100644    Hard   
55612  2019-0605      20191111        299    SF     126774    103819    Hard   
55613  2019-0605      20191111        300     F     126774    106233    Hard   

      tourney_level  best_of  AvgW  Avg

In [120]:
gera_bases_finais()

1-00:37:30
101-00:37:38
201-00:37:45
301-00:37:53
401-00:38:01
501-00:38:09
601-00:38:16
701-00:38:24
801-00:38:32
901-00:38:40
1001-00:38:48
1101-00:38:57
1201-00:39:05
1301-00:39:13
1401-00:39:21
1501-00:39:30
1601-00:39:38
1701-00:39:46
1801-00:39:55
1901-00:40:03
2001-00:40:12
2101-00:40:20
2201-00:40:29
2301-00:40:38
2401-00:40:47
2501-00:40:56
2601-00:41:05
2701-00:41:14
2801-00:41:23
2901-00:41:33
3001-00:41:42
3101-00:41:52
3201-00:42:01
3301-00:42:11
3401-00:42:22
3501-00:42:33
3601-00:42:45
3701-00:42:55
3801-00:43:04
3901-00:43:23
4001-00:43:37
4101-00:43:51
4201-00:44:05
4301-00:44:19
4401-00:44:33
4501-00:44:47
4601-00:45:01
4701-00:45:15
4801-00:45:29
4901-00:45:43
5001-00:46:02
5101-00:46:21
5201-00:46:36
5301-00:46:50
5401-00:47:02
5501-00:47:16
5601-00:47:28
5701-00:47:40
5801-00:47:52
5901-00:48:05
6001-00:48:17
6101-00:48:35
6201-00:48:47
6301-00:49:00
6401-00:49:12
6501-00:49:25
6601-00:49:38
6701-00:49:50
6801-00:50:03
6901-00:50:16
7001-00:50:29
7101-00:50:42
7201

In [118]:
df_ABC = pd.read_excel (r'C:\Giba\Dados\_A_Giba\\Documentos\Adm\PUC_MG\Python\base_completa_XX.xlsx')
print(df_ABC)

     Unnamed: 0 tourney_id  tourney_date  match_num round surface  \
0             0   2001-339      20010101          1   R32    Hard   
1             0   2001-451      20010101          1   R32    Hard   
2             0   2001-891      20010101          1   R32    Hard   
3             0   2001-451      20010101          2   R32    Hard   
4             0   2001-891      20010101          2   R32    Hard   
..          ...        ...           ...        ...   ...     ...   
296           0   2001-506      20010219         11   R32    Clay   
297           0   2001-402      20010219         12   R64    Hard   
298           0   2001-407      20010219         12   R32    Hard   
299           0   2001-506      20010219         12   R32    Clay   
300           0   2001-407      20010219         13   R32    Hard   

    tourney_level  best_of  AvgW  AvgL  ...  \
0               A        3  1.20  3.55  ...   
1               A        3  1.30  2.77  ...   
2               A        3  3.

In [8]:
ajusta_dataframes_resultados_agregados()

      tourney_id  tourney_date  match_num round  winner_id  loser_id surface  \
0       2000-339      20000103          1   R32     102358    103096    Hard   
1       2000-451      20000103          1   R32     103017    101543    Hard   
2       2000-891      20000103          1   R32     103420    102338    Hard   
3       2000-339      20000103          2   R32     103819    102533    Hard   
4       2000-451      20000103          2   R32     101750    102730    Hard   
...          ...           ...        ...   ...        ...       ...     ...   
55609  2019-0605      20191111        296    RR     104745    126774    Hard   
55610  2019-0605      20191111        297    RR     104745    106421    Hard   
55611  2019-0605      20191111        298    SF     106233    100644    Hard   
55612  2019-0605      20191111        299    SF     126774    103819    Hard   
55613  2019-0605      20191111        300     F     126774    106233    Hard   

      tourney_level  best_of  AvgW  Avg

In [49]:
gera_dataframes_resultados_agregados()

--------------------------------------------
1998
23:55:04
3591
--------------------------------------------
1-23:55:04
101-23:55:25
201-23:55:47
301-23:56:10
401-23:56:32
501-23:56:55
601-23:57:18
701-23:57:41
801-23:58:04
901-23:58:27
1001-23:58:51
1101-23:59:15
1201-23:59:39
1301-00:00:04
1401-00:00:29
1501-00:00:54
1601-00:01:19
1701-00:01:46
1801-00:02:13
1901-00:02:39
2001-00:03:06
2101-00:03:34
2201-00:04:02
2301-00:04:30
2401-00:04:59
2501-00:05:27
2601-00:05:57
2701-00:06:27
2801-00:06:58
2901-00:07:28
3001-00:07:59
3101-00:08:31
3201-00:09:03
3301-00:09:35
3401-00:10:07
3501-00:10:40
    player_id tourney_id tourney_date round match_num match_seq  \
0      102019   1998-339     19980105   R32         6         1   
1      103066   1998-301     19980112   R16        17         2   
2      102875   1998-580     19980119  R128        62         1   
3      102578   1998-322     19980202   R32        15         1   
4      101926   1998-322     19980202   R16        19         2 

--------------------------------------------
2000
00:30:35
3378
--------------------------------------------
1-00:30:35
101-00:31:11
201-00:31:48
301-00:32:26
401-00:33:02
501-00:33:40
601-00:34:19
701-00:34:57
801-00:35:35
901-00:36:15
1001-00:36:54
1101-00:37:35
1201-00:38:15
1301-00:38:56
1401-00:39:37
1501-00:40:18
1601-00:41:00
1701-00:41:42
1801-00:42:25
1901-00:43:08
2001-00:43:52
2101-00:44:35
2201-00:45:19
2301-00:46:04
2401-00:46:49
2501-00:47:35
2601-00:48:25
2701-00:49:22
2801-00:50:14
2901-00:51:07
3001-00:52:00
3101-00:52:49
3201-00:53:38
3301-00:54:28
    player_id tourney_id tourney_date round match_num match_seq  \
0      102019   1998-339     19980105   R32         6         1   
1      102875   1998-580     19980119  R128        62         1   
2      102578   1998-322     19980202   R32        15         1   
3      101926   1998-322     19980202   R16        19         2   
4      101841   1998-495     19980209   R32        11         1   
..        ...        ... 

--------------------------------------------
2002
01:25:31
3213
--------------------------------------------
1-01:25:31
101-01:26:26
201-01:27:26
301-01:28:22
401-01:29:20
501-01:30:19
601-01:31:19
701-01:32:20
801-01:33:20
901-01:34:18
1001-01:35:19
1101-01:36:19
1201-01:37:23
1301-01:38:26
1401-01:39:28
1501-01:40:29
1601-01:41:29
1701-01:42:31
1801-01:43:32
1901-01:44:33
2001-01:45:37
2101-01:46:46
2201-01:48:05
2301-01:49:37
2401-01:51:15
2501-01:52:30
2601-01:53:42
2701-01:54:51
2801-01:56:04
2901-01:57:14
3001-01:58:21
3101-01:59:29
3201-02:00:42
    player_id tourney_id tourney_date round match_num match_seq  \
0      102019   1998-339     19980105   R32         6         1   
1      102578   1998-322     19980202   R32        15         1   
2      101926   1998-322     19980202   R16        19         2   
3      101841   1998-495     19980209   R32        11         1   
4      103164   1998-568     19980209   R32        12         1   
..        ...        ...          ...  

--------------------------------------------
2004
02:39:09
3288
--------------------------------------------
1-02:39:09
101-02:40:15
201-02:41:21
301-02:42:28
401-02:43:35
501-02:44:43
601-02:45:51
701-02:47:00
801-02:48:09
901-02:49:19
1001-02:50:28
1101-02:51:38
1201-02:52:49
1301-02:54:00
1401-02:55:12
1501-02:56:24
1601-02:57:36
1701-02:58:49
1801-03:00:02
1901-03:01:16
2001-03:02:30
2101-03:03:45
2201-03:05:01
2301-03:06:16
2401-03:07:34
2501-03:08:50
2601-03:10:07
2701-03:11:24
2801-03:12:42
2901-03:14:00
3001-03:15:19
3101-03:16:37
3201-03:17:57
     player_id tourney_id tourney_date round match_num match_seq  \
0       102019   1998-339     19980105   R32         6         1   
1       102578   1998-322     19980202   R32        15         1   
2       101926   1998-322     19980202   R16        19         2   
3       101841   1998-495     19980209   R32        11         1   
4       103164   1998-568     19980209   R32        12         1   
...        ...        ...        

--------------------------------------------
2006
10:58:48
3267
--------------------------------------------
1-10:58:48
101-11:00:17
201-11:01:45
301-11:03:13
401-11:04:42
501-11:06:11
601-11:07:43
701-11:09:13
801-11:10:43
901-11:12:17
1001-11:13:48
1101-11:15:19
1201-11:16:52
1301-11:18:24
1401-11:19:57
1501-11:21:37
1601-11:23:22
1701-11:25:14
1801-11:27:00
1901-11:28:41
2001-11:30:31
2101-11:32:11
2201-11:33:48
2301-11:35:42
2401-11:37:27
2501-11:39:23
2601-11:41:06
2701-11:42:46
2801-11:44:25
2901-11:46:05
3001-11:47:46
3101-11:49:32
3201-11:51:23
     player_id tourney_id tourney_date round match_num match_seq  \
0       102019   1998-339     19980105   R32         6         1   
1       102578   1998-322     19980202   R32        15         1   
2       101926   1998-322     19980202   R16        19         2   
3       101841   1998-495     19980209   R32        11         1   
4       103164   1998-568     19980209   R32        12         1   
...        ...        ...        

...

--------------------------------------------
2012
17:14:49
3010
--------------------------------------------
1-17:14:49
101-17:17:11
201-17:19:27
301-17:22:45
401-17:26:33
501-17:28:49
601-17:31:04
701-17:33:21
801-17:35:41
901-17:37:57
1001-17:40:13
1101-17:42:32
1201-17:44:54
1301-17:47:13
1401-17:49:32
1501-17:52:07
1601-17:54:39
1701-17:57:18
1801-17:59:59
1901-18:02:40
2001-18:05:11
2101-18:07:29
2201-18:09:49
2301-18:12:09
2401-18:14:30
2501-18:17:00
2601-18:19:29
2701-18:22:01
2801-18:24:29
2901-18:27:10
3001-18:29:54
     player_id                      tourney_id tourney_date round match_num  \
0       102019                        1998-339     19980105   R32         6   
1       102578                        1998-322     19980202   R32        15   
2       101926                        1998-322     19980202   R16        19   
3       101841                        1998-495     19980209   R32        11   
4       103164                        1998-568     19980209   R32        1

--------------------------------------------
2014
19:45:52
2901
--------------------------------------------
1-19:45:52
101-19:48:14
201-19:50:37
301-19:52:59
401-19:55:24
501-19:57:48
601-20:00:15
701-20:02:42
801-20:05:07
901-20:07:33
1001-20:10:00
1101-20:12:27
1201-20:14:54
1301-20:17:22
1401-20:19:55
1501-20:53:20
1601-20:55:59
1701-20:58:44
1801-21:01:44
1901-21:04:21
2001-21:07:13
2101-21:09:57
2201-21:12:32
2301-21:15:07
2401-21:17:42
2501-21:20:16
2601-21:22:53
2701-21:25:29
2801-21:28:06
2901-21:30:43
     player_id tourney_id tourney_date round match_num match_seq  \
0       102019   1998-339     19980105   R32         6         1   
1       102578   1998-322     19980202   R32        15         1   
2       101926   1998-322     19980202   R16        19         2   
3       101841   1998-495     19980209   R32        11         1   
4       103164   1998-568     19980209   R32        12         1   
...        ...        ...          ...   ...       ...       ...   
2108   

--------------------------------------------
2016
00:00:39
2941
--------------------------------------------
1-00:00:39
101-00:03:17
201-00:05:53
301-00:08:33
401-00:11:10
501-00:13:49
601-00:16:31
701-00:19:12
801-00:21:54
901-00:24:39
1001-00:27:21
1101-00:30:03
1201-00:32:46
1301-00:35:29
1401-00:38:12
1501-00:40:56
1601-00:43:41
1701-00:46:26
1801-00:49:11
1901-00:51:56
2001-00:54:42
2101-00:57:30
2201-01:00:17
2301-01:03:04
2401-01:05:51
2501-01:08:40
2601-01:11:31
2701-01:14:20
2801-01:17:08
2901-01:19:55
     player_id                      tourney_id tourney_date round match_num  \
0       102019                        1998-339     19980105   R32         6   
1       102578                        1998-322     19980202   R32        15   
2       101926                        1998-322     19980202   R16        19   
3       101841                        1998-495     19980209   R32        11   
4       103164                        1998-568     19980209   R32        12   
...      

--------------------------------------------
2018
02:50:58
2889
--------------------------------------------
1-02:50:58
101-02:53:51
201-02:56:44
301-02:59:38
401-03:02:32
501-03:05:28
601-03:08:25
701-03:11:21
801-03:14:17
901-03:17:13
1001-03:20:10
1101-03:23:08
1201-10:46:26
1301-10:49:31
1401-10:53:19
1501-10:56:58
1601-11:00:19
1701-11:03:28
1801-11:06:45
1901-11:09:51
2001-11:12:55
2101-11:15:59
2201-11:19:14
2301-11:23:36
2401-11:27:26
2501-11:31:05
2601-11:34:14
2701-11:37:22
2801-11:40:31
     player_id                      tourney_id tourney_date round match_num  \
0       102019                        1998-339     19980105   R32         6   
1       102578                        1998-322     19980202   R32        15   
2       101926                        1998-322     19980202   R16        19   
3       101841                        1998-495     19980209   R32        11   
4       103164                        1998-568     19980209   R32        12   
...        ...         

13:13:45
OK


In [49]:
# Use pandas. DataFrame. append() to add a list as a row
df = pd.DataFrame([[1, 2], [3, 4]], columns = ["a", "b"])
print(df)
to_append = [5, 6]
a_series = pd.Series(to_append, index = df. columns)
df = df.append(a_series, ignore_index=True)
print(df)

   a  b
0  1  2
1  3  4
   a  b
0  1  2
1  3  4
2  5  6


In [74]:
merge_atp_and_odds_info()

 
2001


C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Lista de resultados contém nulos
      ATP   Location                           Tournament       Date  \
558    15   Acapulco                     Abierto Mexicano 2001-02-26   
569    15   Acapulco                     Abierto Mexicano 2001-02-26   
570    15   Acapulco                     Abierto Mexicano 2001-02-26   
1082   27    Houston  U.S. Men's Clay Court Championships 2001-04-30   
1088   27    Houston  U.S. Men's Clay Court Championships 2001-04-30   
1378   33      Paris                          French Open 2001-05-28   
1929   45  Kitzbuhel                        Generali Open 2001-07-23   
1951   45  Kitzbuhel                        Generali Open 2001-07-23   

                  Series    Court Surface          Round  Best of  \
558   International Gold  Outdoor    Clay      1st Round        3   
569   International Gold  Outdoor    Clay      2nd Round        3   
570   International Gold  Outdoor    Clay  Quarterfinals        3   
1082       International  Outdoor    Clay 

In [21]:
gera_lista_torneios_arquivo_results_em_EXCEL()

2001


C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
      Year                       tourney_id                  tourney_name
0     2001                         2001-301                      Auckland
1     2001                         2001-306                   St. Poelten
2     2001                         2001-308                        Munich
3     2001                         2001-311                  Queen's Club
4     2001                         2001-314                        Gstaad
...    ...                              ...                           ...
2755  2019  2019-M-DC-2019-QLS-M-JPN-CHN-01  Davis Cup QLS R1: JPN vs CHN
2756  2019  2019-M-DC-2019-QLS-M-KAZ-POR-01  Davis Cup QLS R1: KAZ vs POR
2757  2019  2019-M-DC-2019-QLS-M-SRB-UZB-01  Davis Cup QLS R1: SRB vs UZB
2758  2019  2019-M-DC-2019-QLS-M-SUI-RUS-01  Davis Cup QLS R1: SUI vs RUS
2759  2019  2019-M-DC-2019-QLS-M-SWE-COL-01  Davis Cup QLS R1: SWE vs COL

[2760 rows x 3 column

In [68]:
gera_lista_torneios_arquivo_ODDS_em_EXCEL()

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [19]:
print(obtem_dataframe_ODDS('2001'))

      ATP  Location          Tournament       Date         Series    Court  \
0       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
1       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
2       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
3       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
4       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
...   ...       ...                 ...        ...            ...      ...   
2958   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2959   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2960   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2961   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2962   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   

     Surface        Round  Best of         Winner  ...    Comme

In [15]:
print(obtem_dataframe_resultados(2001))

     tourney_id                 tourney_name surface draw_size tourney_level  \
1      2001-301                     Auckland    Hard        32             A   
2      2001-301                     Auckland    Hard        32             A   
3      2001-301                     Auckland    Hard        32             A   
4      2001-301                     Auckland    Hard        32             A   
5      2001-301                     Auckland    Hard        32             A   
...         ...                          ...     ...       ...           ...   
3303  2001-D081  Davis Cup G1 PO: ZIM vs SLO    Hard         4             D   
3304   2001-615                   Dusseldorf    Clay        64             A   
3305   2001-615                   Dusseldorf    Clay        64             A   
3306   2001-615                   Dusseldorf    Clay        64             A   
3307   2001-615                   Dusseldorf    Clay        64             A   

     tourney_date match_num winner_id w

C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [20]:
gera_lista_jogadores_arquivo_ODDS_em_EXCEL()

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [134]:
gera_lista_jogadores_arquivo_results_em_EXCEL()

2001


C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gilberto\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
OK


In [32]:
obtem_dataframe_ODDS('2001')

     ATP Location           Tournament       Date              Series  \
588   16    Dubai  Dubai Championships 2001-02-26  International Gold   

       Court Surface      Round  Best of    Winner  ... CBL  GBW GBL  IWW  \
588  Outdoor    Hard  1st Round        3  Pavel A.  ... NaN  NaN NaN  NaN   

     IWL  SBW  SBL  Year  AvgW  AvgL  
588  NaN  NaN  NaN  2001   NaN   NaN  

[1 rows x 37 columns]


,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,CBL,GBW,GBL,IWW,IWL,SBW,SBL,Year,AvgW,AvgL
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Clement A.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Goldstein P.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Haas T.,...,NaN,1.12,4.00,1.10,3.5,NaN,NaN,2001,1.11,3.75
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Henman T.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Hewitt L.,...,3.30,1.12,3.80,NaN,NaN,NaN,NaN,2001,1.20,3.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2958,69,Sydney,Masters Cup,2001-11-12,Masters Cup,Indoor,Hard,Round Robin,3,Kafelnikov Y.,...,2.45,1.45,2.45,1.35,2.5,1.500,2.50,2001,1.46,2.48
2959,69,Sydney,Masters Cup,2001-11-12,Masters Cup,Indoor,Hard,Round Robin,3,Kafelnikov Y.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN
2960,69,Sydney,Masters Cup,2001-11-12,Masters Cup,Indoor,Hard,Semifinals,3,Grosjean S.,...,2.05,1.70,1.95,1.70,1.8,1.667,2.10,2001,1.69,1.98
2961,69,Sydney,Masters Cup,2001-11-12,Masters Cup,Indoor,Hard,Semifinals,3,Hewitt L.,...,2.55,1.20,3.80,1.20,3.3,1.300,3.20,2001,1.29,3.21


In [34]:
for ano in range(2001,2020):
    obtem_dataframe_ODDS(str(ano))

     ATP Location           Tournament       Date              Series  \
588   16    Dubai  Dubai Championships 2001-02-26  International Gold   

       Court Surface      Round  Best of    Winner  ... CBL  GBW GBL  IWW  \
588  Outdoor    Hard  1st Round        3  Pavel A.  ... NaN  NaN NaN  NaN   

     IWL  SBW  SBL  Year  AvgW  AvgL  
588  NaN  NaN  NaN  2001   NaN   NaN  

[1 rows x 37 columns]
     ATP Location                           Tournament       Date  \
535   15    Dubai                  Dubai Championships 2002-02-25   
997   25  Houston  U.S. Men's Clay Court Championships 2002-04-22   

                 Series    Court Surface      Round  Best of     Winner  ...  \
535  International Gold  Outdoor    Hard  1st Round        3  Henman T.  ...   
997       International  Outdoor    Clay  1st Round        3   Coria G.  ...   

    CBL  GBW GBL  IWW  IWL  SBW  SBL  Year  AvgW  AvgL  
535 NaN  NaN NaN  NaN  NaN  NaN  NaN  2002   NaN   NaN  
997 NaN  NaN NaN  NaN  NaN  NaN  N

In [3]:
# glb_workbook = xlrd.open_workbook(glb_diretorio_base + glb_arquivo_configuracao)
# worksheet = glb_workbook.sheet_by_name('Configuracoes')
# df = pd.read_excel(glb_diretorio_base + glb_arquivo_configuracao, sheet_name='Redes', skiprows = 0, index_col=None)
df = pd.read_csv("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\ODDS_ZZZZ_2001.csv", sep=',', delimiter=None, header='infer', names=None, index_col=None)

In [4]:
print(df)

   ATP Location           Tournament        Date              Series    Court  \
0   16    Dubai  Dubai Championships  2001-02-26  International Gold  Outdoor   

  Surface      Round  Best of    Winner  ... CBL  GBW GBL  IWW  IWL  SBW  SBL  \
0    Hard  1st Round        3  Pavel A.  ... NaN  NaN NaN  NaN  NaN  NaN  NaN   

   Year  AvgW  AvgL  
0  2001   NaN   NaN  

[1 rows x 37 columns]


In [7]:
dfx = df[['Year', 'Location', 'Tournament' , 'Date', 'Winner', 'Loser', 'W1','L1','W2','L2','W3','L3','W4','L4','W5', 'L5']].copy()

In [8]:
print(dfx)

   Year Location           Tournament        Date    Winner          Loser  \
0  2001    Dubai  Dubai Championships  2001-02-26  Pavel A.  Bahrouzyan O.   

    W1   L1   W2   L2  W3  L3  W4  L4  W5  L5  
0  6.0  1.0  6.0  2.0 NaN NaN NaN NaN NaN NaN  


In [10]:
for ano in range(2001,2020):
    df = pd.read_csv("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\ODDS_ZZZZ_" + str(ano) + ".csv", sep=',', delimiter=None, header='infer', names=None, index_col=None)
    dfx = df[['Year', 'Location', 'Tournament' , 'Date', 'Winner', 'Loser', 'W1','L1','W2','L2','W3','L3','W4','L4','W5', 'L5']].copy()
    if ano == 2001:
       dfy = dfx
    else:
       dfy=pd.concat([dfy,dfx])  
print(dfy)    
dfy.to_excel("C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\Python\\crazy_players.xlsx")  

   Year        Location                           Tournament        Date  \
0  2001           Dubai                  Dubai Championships  2001-02-26   
0  2002           Dubai                  Dubai Championships  2002-02-25   
1  2002         Houston  U.S. Men's Clay Court Championships  2002-04-22   
0  2003            Doha               Qatar Exxon Mobil Open  2002-12-30   
1  2003            Doha               Qatar Exxon Mobil Open  2003-01-01   
2  2003          Dubai                   Dubai Championships  2003-02-24   
0  2004          Dubai                   Dubai Championships  2004-03-01   
0  2005          Dubai            Dubai Duty Free Men's Open  2005-02-21   
0  2006          Dubai            Dubai Tennis Championships  2006-02-27   
1  2006          Gstaad                  Allianz Suisse Open  2006-07-11   
0  2007     Monte Carlo                  Monte Carlo Masters  2007-04-16   
1  2007          Dubai            Dubai Tennis Championships  2007-02-26   
2  2007     

In [24]:
df_de_para_players = pd.read_excel('C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\TCC\\Meu_Trabalho\\players_de_para.xlsx', sheet_name='DE_PARA', skiprows = 0, index_col=None)

In [23]:
print(df)

           Player_Name  Player_ID  Obs
0        Al Ghareeb M.     103600  NOK
1     Al Khulaifi N.G.     102285  NOK
2         Al Mutawa J.     106325  NOK
3        Al-Alawi S.K.     102956  NOK
4        Al-Ghareeb M.     103600  NOK
...                ...        ...  ...
1375         Muster T.     101404   OK
1376     Gustafsson M.     101320   OK
1377          Pozzi G.     101150   OK
1378         Agenor R.     101086   OK
1379         Zverev A.     100644   OK

[1380 rows x 3 columns]


In [25]:
df_odds = obtem_dataframe_ODDS('2001')

     ATP Location           Tournament       Date              Series  \
588   16    Dubai  Dubai Championships 2001-02-26  International Gold   

       Court Surface      Round  Best of    Winner  ... CBL  GBW GBL  IWW  \
588  Outdoor    Hard  1st Round        3  Pavel A.  ... NaN  NaN NaN  NaN   

     IWL  SBW  SBL  Year  AvgW  AvgL  
588  NaN  NaN  NaN  2001   NaN   NaN  

[1 rows x 37 columns]


In [26]:
print(df_odds)

      ATP  Location          Tournament       Date         Series    Court  \
0       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
1       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
2       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
3       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
4       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
...   ...       ...                 ...        ...            ...      ...   
2958   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2959   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2960   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2961   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2962   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   

     Surface        Round  Best of         Winner  ...   CBL   

In [30]:
df_1 = pd.merge(df_odds, df_de_para_players, how='left', on=None, left_on='Winner', right_on='Player_Name',
         left_index=False, right_index=False, sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [31]:
print(df_1)

      ATP  Location          Tournament       Date         Series    Court  \
0       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
1       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
2       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
3       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
4       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
...   ...       ...                 ...        ...            ...      ...   
2958   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2959   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2960   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2961   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2962   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   

     Surface        Round  Best of         Winner  ...   IWW  I

In [33]:
df_NaN = df_1[df_1['Player_Name'].isnull()]

In [34]:
print(df_NaN)

Empty DataFrame
Columns: [ATP, Location, Tournament, Date, Series, Court, Surface, Round, Best of, Winner, Loser, WRank, LRank, W1, L1, W2, L2, W3, L3, W4, L4, W5, L5, Wsets, Lsets, Comment, CBW, CBL, GBW, GBL, IWW, IWL, SBW, SBL, Year, AvgW, AvgL, Player_Name, Player_ID, Obs]
Index: []

[0 rows x 40 columns]


In [35]:
del df_1["Player_Name"]

In [36]:
print(df_1)

      ATP  Location          Tournament       Date         Series    Court  \
0       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
1       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
2       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
3       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
4       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
...   ...       ...                 ...        ...            ...      ...   
2958   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2959   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2960   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2961   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2962   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   

     Surface        Round  Best of         Winner  ...   GBL   

In [37]:
print(df_NaN.count)

<bound method DataFrame.count of Empty DataFrame
Columns: [ATP, Location, Tournament, Date, Series, Court, Surface, Round, Best of, Winner, Loser, WRank, LRank, W1, L1, W2, L2, W3, L3, W4, L4, W5, L5, Wsets, Lsets, Comment, CBW, CBL, GBW, GBL, IWW, IWL, SBW, SBL, Year, AvgW, AvgL, Player_Name, Player_ID, Obs]
Index: []

[0 rows x 40 columns]>


In [38]:
print(df_NaN.empty)

True


In [40]:
df_1.rename(columns={'Player_ID': 'Winner_ID'}, inplace=True)

In [41]:
print(df_1)

      ATP  Location          Tournament       Date         Series    Court  \
0       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
1       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
2       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
3       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
4       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
...   ...       ...                 ...        ...            ...      ...   
2958   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2959   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2960   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2961   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2962   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   

     Surface        Round  Best of         Winner  ...   GBL   

In [2]:
df_de_para_tourneys = pd.read_excel('C:\\Giba\Dados\\_A_Giba\\Documentos\\Adm\\PUC_MG\\TCC\\Meu_Trabalho\\tourneys_de_para.xlsx', sheet_name='DE_PARA', skiprows = 0, index_col=None)

In [60]:
df_3 = pd.merge(df_odds, df_de_para_tourneys, how='left', on=None, left_on=['Year','ATP', 'Location','Tournament'], right_on=['Year','ATP', 'Location','Tournament'],
         left_index=False, right_index=False, sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [58]:
df_3 = pd.merge(df_odds, df_de_para_tourneys, on=['Year','ATP', 'Location','Tournament'])

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [19]:
df_odds = obtem_dataframe_ODDS(str(2001))

In [20]:
merged_df = df_odds.merge(df_de_para_tourneys, how='left', left_on=['Year', 'ATP', 'Location','Tournament'], right_on=['Year', 'ATP', 'Location','Tournament'])

In [21]:
print(merged_df)

      ATP  Location          Tournament       Date         Series    Court  \
0       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
1       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
2       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
3       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
4       1  Adelaide  AAPT Championships 2001-01-01  International  Outdoor   
...   ...       ...                 ...        ...            ...      ...   
2958   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2959   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2960   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2961   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   
2962   69    Sydney         Masters Cup 2001-11-12    Masters Cup   Indoor   

     Surface        Round  Best of         Winner  ...   GBW   

In [46]:
df_results = obtem_dataframe_resultados('2019')

In [53]:
print(df_results)

                           tourney_id                     tourney_name  \
58                          2019-0451                             Doha   
27                          2019-M020                         Brisbane   
57                          2019-0451                             Doha   
85                          2019-0891                             Pune   
26                          2019-M020                         Brisbane   
...                               ...                              ...   
2656  2019-M-DC-2019-FLS-M-RUS-CAN-01  Davis Cup Finals SF: RUS vs CAN   
2653  2019-M-DC-2019-FLS-M-GBR-ESP-01  Davis Cup Finals SF: GBR vs ESP   
2657  2019-M-DC-2019-FLS-M-RUS-CAN-01  Davis Cup Finals SF: RUS vs CAN   
2650  2019-M-DC-2019-FLS-M-CAN-ESP-01   Davis Cup Finals F: CAN vs ESP   
2651  2019-M-DC-2019-FLS-M-CAN-ESP-01   Davis Cup Finals F: CAN vs ESP   

     surface draw_size tourney_level tourney_date  match_num winner_id  \
58      Hard        32             A 